# Run this in Kaggle

📘 1️⃣ Project Introduction

# 🔍 Altered Fingerprint Detection & Restoration for Forensic Analysis

This notebook builds two complete deep learning pipelines:

1) **Fingerprint Alteration Detection (Real vs Altered)**
   - Dataset split by subject (prevents leakage)
   - ResNet50 classifier with weighted loss
   - Validation/test evaluation
   - Random prediction visualizations

2) **Fingerprint Restoration using Pix2Pix GAN**
   - Pairs altered → real images
   - U-Net generator + PatchGAN discriminator
   - Adversarial + L1 training loop
   - PSNR, SSIM, MSE metrics
   - Artifact visualization and monitoring

Additionally, we create:
- A small downloadable subset for the Colab-based web application  
- Saved model weights for both classifier and GAN  

This notebook runs end-to-end on Kaggle GPU.


📘 Dataset Attachment & Setup Instructions

## 📁 Dataset Setup: SOCOFing Fingerprint Dataset

Before running any code in this notebook:

1. Go to the dataset page:  
   https://www.kaggle.com/datasets/ruizgara/socofing

2. Click **“Download”** or **“Add to your Kaggle Datasets”** — whichever applies.  
   If you download manually, upload the resulting ZIP file to Kaggle (via “Upload Data”) and extract  
   so that the following folder structure is available:

   /kaggle/input/socofing/SOCOFing/  
       ├── Real/  
       └── Altered/  
            ├── Altered-Easy/  
            ├── Altered-Medium/  
            └── Altered-Hard/

3. If using the “Add to your Kaggle Datasets” method, ensure that the dataset alias:  
   **ruizgara/socofing** is attached to your notebook.

4. Confirm that inside the notebook the following paths exist:  
   ```text
   /kaggle/input/socofing/SOCOFing/Real  
   /kaggle/input/socofing/SOCOFing/Altered/Altered-Easy  
   /kaggle/input/socofing/SOCOFing/Altered/Altered-Medium  
   /kaggle/input/socofing/SOCOFing/Altered/Altered-Hard


In [ ]:
!pip install -U pip --quiet
!pip install numpy==1.26.4 scipy==1.14.1 scikit-learn==1.6.0 matplotlib==3.9.2 \
              opencv-python==4.10.0.84 pillow==10.4.0 pandas==2.2.3 tqdm==4.66.4 \
              albumentations==1.4.13 scikit-image==0.24.0 lpips==0.1.4 \
              torch torchvision torchaudio --quiet


📘 2️⃣ Environment Setup

This cell installs all necessary libraries:

- NumPy, SciPy, scikit-learn → scientific computing  
- OpenCV, Pillow → image processing  
- Albumentations → augmentations  
- LPIPS → perceptual loss (optional)  
- PyTorch + torchvision → model training  
- tqdm → progress bars  

Once installed, we verify versions and confirm GPU usage.


In [ ]:
import numpy, scipy, sklearn, matplotlib, torch
print("NumPy:", numpy.__version__)
print("SciPy:", scipy.__version__)
print("sklearn:", sklearn.__version__)
print("Matplotlib:", matplotlib.__version__)
print("Torch:", torch.__version__)


📘 3️⃣ Dataset Paths & Basic Setup

The SOCOFing dataset contains:

- Real fingerprints  
- Altered fingerprints (Easy, Medium, Hard)  
  Types: CR (central rotation), OB (obliteration), ZW (z-warp)

We collect all BMP file paths and label them:
- real  
- altered_easy  
- altered_medium  
- altered_hard  

We also extract subject IDs to ensure proper train/val/test splitting.


In [ ]:
# ===============================
# 1️⃣ SETUP ENVIRONMENT
# ===============================
import os, glob, random, shutil
import numpy as np
import pandas as pd
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models, utils
from tqdm.notebook import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)


In [ ]:
# ===============================
# 2️⃣ DATASET PATHS
# ===============================
base = "/kaggle/input/socofing/SOCOFing"
real_path = f"{base}/Real"
alter_easy = f"{base}/Altered/Altered-Easy"
alter_med  = f"{base}/Altered/Altered-Medium"
alter_hard = f"{base}/Altered/Altered-Hard"

# Collect image paths and labels
def collect_paths(folder, label):
    files = glob.glob(os.path.join(folder, "*.BMP"))
    return [(f, label) for f in files]

data = []
data += collect_paths(real_path, "real")
data += collect_paths(alter_easy, "altered_easy")
data += collect_paths(alter_med, "altered_medium")
data += collect_paths(alter_hard, "altered_hard")

df = pd.DataFrame(data, columns=["path", "label"])
df["subject_id"] = df["path"].apply(lambda x: os.path.basename(x).split("__")[0])
df.sample(5)


📘 4️⃣ Splitting Dataset by Subject ID

We prevent data leakage by splitting based on subject_id.

This ensures:
- Training fingerprints and test fingerprints never belong to the same person  
- More realistic forensic evaluation  


In [ ]:
# Split at subject-level to avoid data leakage
unique_subjects = df["subject_id"].unique()
train_ids, test_ids = train_test_split(unique_subjects, test_size=0.2, random_state=42)
val_ids, test_ids = train_test_split(test_ids, test_size=0.5, random_state=42)

train_df = df[df["subject_id"].isin(train_ids)]
val_df = df[df["subject_id"].isin(val_ids)]
test_df = df[df["subject_id"].isin(test_ids)]

print("Train:", len(train_df), "Val:", len(val_df), "Test:", len(test_df))


In [ ]:
print("Train distribution:")
print(train_df['label'].value_counts())
print("\nValidation distribution:")
print(val_df['label'].value_counts())
print("\nTest distribution:")
print(test_df['label'].value_counts())


📘 5️⃣ Class Weight Calculation

The dataset contains **many more 'real'** samples than altered.

To balance the model:
- Compute class weights  
- Apply weighted cross-entropy loss  

This prevents bias toward predicting "real" every time.


In [ ]:
# ===============================
# ⚖️ CLASS WEIGHTS (Option A – Weighted Loss)
# ===============================
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import torch

# Define the class labels (0 = real, 1 = altered)
classes = np.array([0, 1])

# Compute weights based on the dataset distribution
real_count = len(df[df['label'] == 'real'])
altered_count = len(df) - real_count

print("Real samples:", real_count)
print("Altered samples:", altered_count)

labels = [0]*real_count + [1]*altered_count
weights = compute_class_weight('balanced', classes=classes, y=labels)

# Display computed weights
print("Computed Class Weights →", weights)
# Example output: [4.45, 0.55]  (meaning class 0 gets higher importance)

# Convert to torch tensor for use in loss function later
class_weights = torch.tensor(weights, dtype=torch.float32).to(device)


📘 6️⃣ Sample Visualization from Each Category

This cell shows one random image from:

- Real
- Altered-Easy
- Altered-Medium
- Altered-Hard

Purpose:
Quick sanity check to visually confirm dataset integrity.


In [ ]:
import matplotlib.pyplot as plt
import random

# Randomly pick samples from each class
categories = ['real', 'altered_easy', 'altered_medium', 'altered_hard']
fig, axes = plt.subplots(1, 4, figsize=(12, 3))

for i, cat in enumerate(categories):
    path = random.choice(df[df['label'] == cat]['path'].values)
    img = Image.open(path).convert('L')
    axes[i].imshow(img, cmap='gray')
    axes[i].set_title(cat)
    axes[i].axis('off')

plt.show()


In [ ]:
import matplotlib.pyplot as plt
import random

# Randomly pick samples from each class
categories = ['real', 'altered_easy', 'altered_medium', 'altered_hard']
fig, axes = plt.subplots(1, 4, figsize=(12, 3))

for i, cat in enumerate(categories):
    path = random.choice(df[df['label'] == cat]['path'].values)
    img = Image.open(path).convert('L')
    axes[i].imshow(img, cmap='gray')
    axes[i].set_title(cat)
    axes[i].axis('off')

plt.show()


In [ ]:
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
print("Dataframe shuffled and ready.")


📘 7️⃣ Dataset Class for Classification

We create a PyTorch Dataset class that:

- Loads & resizes fingerprint images  
- Converts to RGB (ResNet50 requirement)  
- Normalizes using ImageNet mean/std  
- Maps labels → 0 (real), 1 (altered)

This class feeds into the DataLoader.


In [ ]:
# ===============================
# 4️⃣ DATASET CLASS
# ===============================
class FingerprintDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform
        self.label_map = {'real': 0, 'altered_easy': 1, 'altered_medium': 1, 'altered_hard': 1}
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(row.path).convert("RGB").resize((224,224))  # <-- changed to RGB
        if self.transform:
            img = self.transform(img)
        label = self.label_map[row.label]
        return img, torch.tensor(label, dtype=torch.long)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_data = FingerprintDataset(train_df, transform)
val_data = FingerprintDataset(val_df, transform)
test_data = FingerprintDataset(test_df, transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32)
test_loader = DataLoader(test_data, batch_size=32)


📘 8️⃣ Classification Model Setup (ResNet50)

We fine-tune ResNet50:

- Pretrained on ImageNet  
- Replace final FC layer → 2 classes  
- Use weighted loss  
- Adam optimizer + LR scheduler  

This model detects whether a fingerprint is real or altered.


In [ ]:
# ===============================
# 5️⃣ CLASSIFICATION MODEL (with Weighted Loss)
# ===============================

# Load a pretrained ResNet50
model = models.resnet50(weights="IMAGENET1K_V1")   # updated param name for PyTorch ≥2.0
model.fc = nn.Linear(model.fc.in_features, 2)      # 2 output classes (real / altered)
model = model.to(device)

# Use the weighted loss computed earlier
criterion = nn.CrossEntropyLoss(weight=class_weights)

# Adam optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)

# Optional learning-rate scheduler (helps stabilize training)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

print("Model, loss, and optimizer ready.")


📘 9️⃣ Training Loop with Early Stopping

Training features:

- Live training accuracy  
- Validation accuracy each epoch  
- Early stopping to prevent overfitting  
- Save best-weight checkpoint to /kaggle/working  

Output:
fingerprint_classifier.pth


In [ ]:
# ===============================
# 6️⃣ TRAINING LOOP (Live Accuracy + Early Stopping)
# ===============================

import time
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score

def train_model(model, train_loader, val_loader, epochs=15, patience=3):
    best_acc = 0
    no_improve = 0
    history = {'train_loss': [], 'train_acc': [], 'val_acc': []}

    for epoch in range(epochs):
        model.train()
        running_loss, correct_train, total_train = 0.0, 0, 0
        start = time.time()

        # ======== TRAINING ========
        for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Training]", leave=False):
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            preds = torch.argmax(outputs, dim=1)
            correct_train += (preds == labels).sum().item()
            total_train += labels.size(0)

        train_acc = correct_train / total_train
        avg_loss = running_loss / len(train_loader)

        # ======== VALIDATION ========
        model.eval()
        preds, true = [], []
        with torch.no_grad():
            for imgs, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} [Validation]", leave=False):
                imgs, labels = imgs.to(device), labels.to(device)
                outputs = model(imgs)
                pred = torch.argmax(outputs, dim=1)
                preds.extend(pred.cpu().numpy())
                true.extend(labels.cpu().numpy())

        val_acc = accuracy_score(true, preds)
        scheduler.step()

        # ======== Logging ========
        history['train_loss'].append(avg_loss)
        history['train_acc'].append(train_acc)
        history['val_acc'].append(val_acc)

        print(f"Epoch {epoch+1:02d}/{epochs} | "
              f"Train Loss: {avg_loss:.4f} | "
              f"Train Acc: {train_acc:.4f} | "
              f"Val Acc: {val_acc:.4f} | "
              f"Time: {(time.time()-start)/60:.2f} min")

        # ======== Early Stopping ========
        if val_acc > best_acc:
            best_acc = val_acc
            no_improve = 0
            torch.save(model.state_dict(), "/kaggle/working/fingerprint_classifier.pth")
            print(f"✅ Best model updated (Val Acc: {best_acc:.4f})")
        else:
            no_improve += 1
            print(f"⚠️ No improvement for {no_improve} epoch(s).")

        if no_improve >= patience:
            print(f"⏹ Early stopping triggered (no improvement for {patience} epochs).")
            break

    print("\nTraining complete. Best validation accuracy:", round(best_acc, 4))
    return history

# Run training
history = train_model(model, train_loader, val_loader, epochs=15, patience=3)


📘 🔟 Evaluate on Test Dataset

We reload the best model and compute:

- Confusion matrix  
- Precision  
- Recall  
- F1-score  
- Class-wise report  

This validates forensic-level performance.


In [ ]:
# ===============================
# 7️⃣ EVALUATION ON TEST SET
# ===============================
from sklearn.metrics import classification_report, confusion_matrix
import torch

y_true, y_pred = [], []
model.load_state_dict(torch.load("/kaggle/working/fingerprint_classifier.pth"))
model.eval()

with torch.no_grad():
    for imgs, labels in test_loader:
        imgs = imgs.to(device)
        outputs = model(imgs)
        preds = outputs.argmax(1).cpu().numpy()
        y_pred.extend(preds)
        y_true.extend(labels.numpy())

print(classification_report(y_true, y_pred, target_names=["real", "altered"]))


📘 1️⃣1️⃣ Random Sample Predictions

Show 6 random fingerprint samples with:

- True label  
- Predicted label  

Useful for debugging & demo screenshots.


In [ ]:
# ===============================
# 🔍 RANDOM SAMPLE PREDICTION DEMO
# ===============================
import matplotlib.pyplot as plt
from PIL import Image
import torch
from torchvision import transforms
import random

# Load model (best checkpoint)
model.load_state_dict(torch.load("/kaggle/working/fingerprint_classifier.pth"))
model.eval()

# Define preprocessing (same as training)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Randomly select samples from test_df
sample_df = test_df.sample(6, random_state=42).reset_index(drop=True)

plt.figure(figsize=(12,6))
for i, row in enumerate(sample_df.itertuples()):
    img = Image.open(row.path).convert("RGB").resize((224,224))
    input_tensor = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(input_tensor)
        pred = torch.argmax(output, dim=1).item()
        label_pred = "Altered" if pred == 1 else "Real"
        label_true = "Altered" if "altered" in row.label else "Real"

    plt.subplot(2, 3, i+1)
    plt.imshow(img, cmap='gray')
    plt.title(f"True: {label_true}\nPred: {label_pred}")
    plt.axis('off')

plt.suptitle("Random Fingerprint Classification Samples", fontsize=14)
plt.tight_layout()
plt.show()

🧾 Current Model Summary

Model file:
fingerprint_classifier.pth

Purpose:
Detects whether a fingerprint is Real or Altered.

Architecture:
ResNet50 (pretrained on ImageNet)
→ fine-tuned for binary classification (real=0, altered=1)

Performance:

Validation Accuracy: 99.85%

Test Accuracy: ~99.96%

Balanced precision and recall for both classes ✅

Early stopping to prevent overfitting ✅

📘 1️⃣2️⃣ GAN Pairing Logic

Pix2Pix GAN requires paired data:
(altered_fingerprint, real_fingerprint)

This cell:
- Extracts subject ID + finger type  
- Matches altered → real pairs intelligently  
- Creates a clean list of exact pairs  


📘 1️⃣3️⃣ GAN Dataset

Creates a dataset tailored for Pix2Pix:

- Loads paired images  
- Converts to grayscale  
- Normalizes to [-1, 1]  
- Outputs (altered_tensor, real_tensor)


In [ ]:
# ===============================
# 8️⃣ CREATE PAIRED DATA (Altered → Real) — Improved Version
# ===============================
import os, glob
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch

# Improved pairing: match both subject ID and finger type
def extract_id_finger(path):
    parts = os.path.basename(path).split("__")
    return "__".join(parts[:2])  # e.g., "100__M_Left_index_finger"

# Build real-image lookup table
real_dict = {extract_id_finger(p): p for p in glob.glob(f"{real_path}/*.BMP")}

# Collect altered→real pairs
pairs = []
for alt_path in (
    glob.glob(f"{alter_easy}/*.BMP") +
    glob.glob(f"{alter_med}/*.BMP") +
    glob.glob(f"{alter_hard}/*.BMP")
):
    # Clean alteration suffixes (e.g., _CR, _OB, _ZW)
    key = extract_id_finger(
        alt_path.replace("_CR", "").replace("_OB", "").replace("_ZW", "")
    )
    if key in real_dict:
        pairs.append((alt_path, real_dict[key]))

print("✅ Paired samples found:", len(pairs))

# ===============================
# 🔧 Dataset for GAN Restoration
# ===============================
class RestorationDataset(Dataset):
    def __init__(self, pairs, transform=None):
        self.pairs = pairs
        self.transform = transform

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        altered_path, real_path = self.pairs[idx]
        a_img = Image.open(altered_path).convert("L").resize((256, 256))
        r_img = Image.open(real_path).convert("L").resize((256, 256))
        if self.transform:
            a_img = self.transform(a_img)
            r_img = self.transform(r_img)
        return a_img, r_img

# Normalization → [-1, 1] range for GAN stability
transform_gan = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Create dataset and dataloader
rest_data = RestorationDataset(pairs, transform_gan)
train_loader_gan = DataLoader(rest_data, batch_size=4, shuffle=True)

print("✅ Restoration dataset ready with", len(rest_data), "samples.")


📘 1️⃣4️⃣ Pix2Pix Architecture

Pix2Pix uses:

- U-Net Generator  
- PatchGAN Discriminator  

Both are recreated here exactly as in the original paper.


In [ ]:
# ===============================
# 9️⃣ PIX2PIX COMPONENTS: Generator (U-Net) + PatchGAN Discriminator
# ===============================
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

# Device should already be set earlier
print("Device:", device)

# ---------------------------
# Helper: weight initialization
# ---------------------------
def init_weights(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
        if getattr(m, 'bias', None) is not None:
            nn.init.constant_(m.bias.data, 0.0)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0.0)

# ---------------------------
# U-Net Generator (grayscale in/out)
# ---------------------------
class UNetGenerator(nn.Module):
    def __init__(self, in_channels=1, out_channels=1, features=64):
        super().__init__()
        # Encoder blocks: Conv -> BN -> LeakyReLU (no BN on first)
        self.down1 = nn.Sequential(
            nn.Conv2d(in_channels, features, 4, 2, 1, bias=True),
            nn.LeakyReLU(0.2, inplace=True)
        )  # 256 -> 128
        self.down2 = nn.Sequential(
            nn.Conv2d(features, features*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features*2),
            nn.LeakyReLU(0.2, inplace=True)
        )  # 128 -> 64
        self.down3 = nn.Sequential(
            nn.Conv2d(features*2, features*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features*4),
            nn.LeakyReLU(0.2, inplace=True)
        )  # 64 -> 32
        self.down4 = nn.Sequential(
            nn.Conv2d(features*4, features*8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features*8),
            nn.LeakyReLU(0.2, inplace=True)
        )  # 32 -> 16
        self.down5 = nn.Sequential(
            nn.Conv2d(features*8, features*8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features*8),
            nn.LeakyReLU(0.2, inplace=True)
        )  # 16 -> 8
        self.down6 = nn.Sequential(
            nn.Conv2d(features*8, features*8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features*8),
            nn.LeakyReLU(0.2, inplace=True)
        )  # 8 -> 4
        self.down7 = nn.Sequential(
            nn.Conv2d(features*8, features*8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features*8),
            nn.LeakyReLU(0.2, inplace=True)
        )  # 4 -> 2
        self.down8 = nn.Sequential(
            nn.Conv2d(features*8, features*8, 4, 2, 1, bias=False),
            nn.ReLU(inplace=True)
        )  # 2 -> 1 (bottleneck)

        # Decoder blocks: ConvTranspose -> BN -> ReLU -> dropout optional -> concat skip
        def up(in_ch, out_ch, dropout=False):
            layers = [nn.ConvTranspose2d(in_ch, out_ch, 4, 2, 1, bias=False),
                      nn.BatchNorm2d(out_ch),
                      nn.ReLU(inplace=True)]
            if dropout:
                layers.append(nn.Dropout(0.5))
            return nn.Sequential(*layers)

        self.up1 = up(features*8, features*8, dropout=True)   # 1 -> 2
        self.up2 = up(features*16, features*8, dropout=True)  # 2 -> 4
        self.up3 = up(features*16, features*8, dropout=True)  # 4 -> 8
        self.up4 = up(features*16, features*8)                # 8 -> 16
        self.up5 = up(features*16, features*4)                # 16 -> 32
        self.up6 = up(features*8, features*2)                 # 32 -> 64
        self.up7 = up(features*4, features)                   # 64 -> 128
        self.up8 = nn.Sequential(
            nn.ConvTranspose2d(features*2, out_channels, 4, 2, 1, bias=True),
            nn.Tanh()
        )  # 128 -> 256

    def forward(self, x):
        e1 = self.down1(x)
        e2 = self.down2(e1)
        e3 = self.down3(e2)
        e4 = self.down4(e3)
        e5 = self.down5(e4)
        e6 = self.down6(e5)
        e7 = self.down7(e6)
        e8 = self.down8(e7)

        d1 = self.up1(e8)
        d1 = torch.cat([d1, e7], dim=1)
        d2 = self.up2(d1)
        d2 = torch.cat([d2, e6], dim=1)
        d3 = self.up3(d2)
        d3 = torch.cat([d3, e5], dim=1)
        d4 = self.up4(d3)
        d4 = torch.cat([d4, e4], dim=1)
        d5 = self.up5(d4)
        d5 = torch.cat([d5, e3], dim=1)
        d6 = self.up6(d5)
        d6 = torch.cat([d6, e2], dim=1)
        d7 = self.up7(d6)
        d7 = torch.cat([d7, e1], dim=1)
        out = self.up8(d7)
        return out

# ---------------------------
# PatchGAN Discriminator
# ---------------------------
class PatchDiscriminator(nn.Module):
    def __init__(self, in_channels=2, features=64):
        super().__init__()
        # in_channels = input(1) concat target/generated(1) = 2
        self.model = nn.Sequential(
            nn.Conv2d(in_channels, features, 4, 2, 1, bias=True),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(features, features*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features*2),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(features*2, features*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features*4),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(features*4, features*8, 4, 1, 1, bias=False),
            nn.BatchNorm2d(features*8),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(features*8, 1, 4, 1, 1, bias=True)  # output one-channel patch map
        )

    def forward(self, x, y):
        # concatenate along channel dim: (B,2,256,256)
        xy = torch.cat([x, y], dim=1)
        return self.model(xy)

# ---------------------------
# Instantiate and init
# ---------------------------
gen = UNetGenerator(in_channels=1, out_channels=1).to(device)
disc = PatchDiscriminator(in_channels=2).to(device)
gen.apply(init_weights)
disc.apply(init_weights)

# ---------------------------
# Losses + optimizers + schedulers
# ---------------------------
bce_loss = nn.BCEWithLogitsLoss()
l1_loss = nn.L1Loss()

lr = 2e-4
opt_g = Adam(gen.parameters(), lr=lr, betas=(0.5, 0.999))
opt_d = Adam(disc.parameters(), lr=lr, betas=(0.5, 0.999))

# Optional schedulers
scheduler_g = torch.optim.lr_scheduler.StepLR(opt_g, step_size=30, gamma=0.5)
scheduler_d = torch.optim.lr_scheduler.StepLR(opt_d, step_size=30, gamma=0.5)

print("Generator params:", sum(p.numel() for p in gen.parameters()))
print("Discriminator params:", sum(p.numel() for p in disc.parameters()))

# ---------------------------
# Smoke test: run a batch through gen + disc to validate shapes
# ---------------------------
batch = next(iter(train_loader_gan))  # expects dataset already created
altered_batch, real_batch = batch
altered_batch = altered_batch.to(device)    # shape (B,1,256,256)
real_batch = real_batch.to(device)

with torch.no_grad():
    fake = gen(altered_batch)               # should be (B,1,256,256)
    d_real = disc(altered_batch, real_batch) # patch map
    d_fake = disc(altered_batch, fake)       # patch map

print("Altered:", altered_batch.shape)
print("Real:", real_batch.shape)
print("Fake:", fake.shape)
print("D_real:", d_real.shape, "D_fake:", d_fake.shape)

# Save initial generator checkpoint (optional)
torch.save(gen.state_dict(), "/kaggle/working/generator_init.pth")
torch.save(disc.state_dict(), "/kaggle/working/discriminator_init.pth")
print("Pix2Pix components ready. Run the training loop next.")


📘 1️⃣5️⃣ Pix2Pix Training Loop

GAN training logic:

- Train discriminator on real vs fake pairs  
- Train generator using adversarial + L1 loss  
- Track PSNR, SSIM, L1  
- Save best generator and checkpoints  

This part runs longer (100 epochs recommended).


In [ ]:
# ===============================
# 🔥 PIX2PIX TRAINING LOOP (Adversarial + L1) with Val Monitoring & Visuals
# ===============================
import time
import os
from torch.utils.data import random_split
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
import numpy as np
import matplotlib.pyplot as plt

# ----- Hyperparams -----
epochs = 100
batch_size = 4               # already used; keep same
lr = 2e-4
lambda_l1 = 100.0            # pix2pix typical
save_every = 5               # save checkpoints every N epochs
viz_every = 5                # visualize every N epochs
patience = 8                 # early stop on val L1 stagnation

# ----- Prepare train/val split -----
val_frac = 0.1
n_total = len(rest_data)
n_val = int(n_total * val_frac)
n_train = n_total - n_val
train_ds, val_ds = random_split(rest_data, [n_train, n_val], generator=torch.Generator().manual_seed(42))
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

print(f"Train pairs: {len(train_ds)} | Val pairs: {len(val_ds)}")

# Ensure optimizers/schedulers are set (they were created earlier but recreate for safety)
opt_g = Adam(gen.parameters(), lr=lr, betas=(0.5, 0.999))
opt_d = Adam(disc.parameters(), lr=lr, betas=(0.5, 0.999))
scheduler_g = torch.optim.lr_scheduler.StepLR(opt_g, step_size=30, gamma=0.5)
scheduler_d = torch.optim.lr_scheduler.StepLR(opt_d, step_size=30, gamma=0.5)

bce_loss = nn.BCEWithLogitsLoss()
l1_loss = nn.L1Loss()

# helpers
def tensor_to_image(tensor):
    # tensor in [-1,1], shape (1,H,W) or (C,H,W) -> uint8 grayscale
    img = tensor.cpu().numpy()
    img = (img * 0.5 + 0.5)  # -> [0,1]
    img = np.clip(img, 0, 1)
    img = (img * 255).astype(np.uint8)
    if img.shape[0] == 1:
        return img[0]
    return np.transpose(img, (1,2,0))

def compute_metrics_batch(fake, real):
    # fake, real: torch tensors shape (B,1,H,W) in [-1,1]
    fake_np = (fake.cpu().numpy() * 0.5 + 0.5)
    real_np = (real.cpu().numpy() * 0.5 + 0.5)
    psnr_vals, ssim_vals = [], []
    for i in range(fake_np.shape[0]):
        f = (fake_np[i,0]*255).astype(np.uint8)
        r = (real_np[i,0]*255).astype(np.uint8)
        try:
            psnr_vals.append(psnr(r, f, data_range=255))
            ssim_vals.append(ssim(r, f, data_range=255))
        except:
            psnr_vals.append(0)
            ssim_vals.append(0)
    return np.mean(psnr_vals), np.mean(ssim_vals)

# training state
best_val_l1 = float('inf')
no_improve = 0

history = {'g_loss': [], 'd_loss': [], 'val_l1': [], 'val_psnr': [], 'val_ssim': []}

for epoch in range(1, epochs+1):
    gen.train(); disc.train()
    t0 = time.time()
    running_g_loss = 0.0
    running_d_loss = 0.0

    for altered, real in train_loader:
        altered = altered.to(device)   # (B,1,256,256)
        real = real.to(device)

        # ---------------------
        # Train Discriminator
        # ---------------------
        opt_d.zero_grad()
        fake = gen(altered)

        # real labels = 1, fake labels = 0
        d_real = disc(altered, real)
        d_fake = disc(altered, fake.detach())

        loss_d_real = bce_loss(d_real, torch.ones_like(d_real))
        loss_d_fake = bce_loss(d_fake, torch.zeros_like(d_fake))
        loss_d = (loss_d_real + loss_d_fake) * 0.5
        loss_d.backward()
        opt_d.step()

        # ---------------------
        # Train Generator
        # ---------------------
        opt_g.zero_grad()
        fake = gen(altered)
        d_fake_for_g = disc(altered, fake)
        loss_g_gan = bce_loss(d_fake_for_g, torch.ones_like(d_fake_for_g))
        loss_g_l1 = l1_loss(fake, real) * lambda_l1
        loss_g = loss_g_gan + loss_g_l1
        loss_g.backward()
        opt_g.step()

        running_g_loss += loss_g.item()
        running_d_loss += loss_d.item()

    # step schedulers
    scheduler_g.step()
    scheduler_d.step()

    avg_g_loss = running_g_loss / len(train_loader)
    avg_d_loss = running_d_loss / len(train_loader)

    # ---- validation (compute average L1, PSNR, SSIM) ----
    gen.eval(); disc.eval()
    val_l1s, val_psnrs, val_ssims = [], [], []
    with torch.no_grad():
        for altered, real in val_loader:
            altered = altered.to(device); real = real.to(device)
            fake = gen(altered)
            val_l1s.append(l1_loss(fake, real).item())
            psnr_b, ssim_b = compute_metrics_batch(fake, real)
            val_psnrs.append(psnr_b); val_ssims.append(ssim_b)

    mean_val_l1 = float(np.mean(val_l1s))
    mean_val_psnr = float(np.mean(val_psnrs))
    mean_val_ssim = float(np.mean(val_ssims))

    history['g_loss'].append(avg_g_loss)
    history['d_loss'].append(avg_d_loss)
    history['val_l1'].append(mean_val_l1)
    history['val_psnr'].append(mean_val_psnr)
    history['val_ssim'].append(mean_val_ssim)

    print(f"Epoch {epoch:03d}/{epochs} | G_loss: {avg_g_loss:.4f} | D_loss: {avg_d_loss:.4f} | "
          f"Val L1: {mean_val_l1:.6f} | PSNR: {mean_val_psnr:.3f} | SSIM: {mean_val_ssim:.4f} | Time: {(time.time()-t0)/60:.2f} min")

    # save checkpoints
    if epoch % save_every == 0:
        torch.save(gen.state_dict(), f"/kaggle/working/generator_epoch{epoch}.pth")
        torch.save(disc.state_dict(), f"/kaggle/working/discriminator_epoch{epoch}.pth")
        print(f"🔖 Saved checkpoint at epoch {epoch}")

    # save best by val L1 (lower better)
    if mean_val_l1 < best_val_l1:
        best_val_l1 = mean_val_l1
        no_improve = 0
        torch.save(gen.state_dict(), "/kaggle/working/generator_best.pth")
        print(f"✅ New best generator (val L1: {best_val_l1:.6f})")
    else:
        no_improve += 1
        print(f"⚠️ No improvement for {no_improve} epoch(s) (best L1: {best_val_l1:.6f})")

    # visualize some samples occasionally
    if epoch % viz_every == 0 or epoch == 1:
        gen.eval()
        altered_sample, real_sample = next(iter(val_loader))
        with torch.no_grad():
            fake_sample = gen(altered_sample.to(device)).cpu()
        altered_sample = altered_sample.cpu()
        real_sample = real_sample.cpu()

        n_show = min(3, altered_sample.shape[0])
        plt.figure(figsize=(9,3))
        for i in range(n_show):
            a = tensor_to_image(altered_sample[i])
            f = tensor_to_image(fake_sample[i])
            r = tensor_to_image(real_sample[i])
            plt.subplot(3, n_show, i+1); plt.imshow(a, cmap='gray'); plt.title("Altered"); plt.axis('off')
            plt.subplot(3, n_show, n_show+i+1); plt.imshow(f, cmap='gray'); plt.title("Restored"); plt.axis('off')
            plt.subplot(3, n_show, 2*n_show+i+1); plt.imshow(r, cmap='gray'); plt.title("Real"); plt.axis('off')
        plt.suptitle(f"Epoch {epoch} samples (Altered → Restored → Real)")
        plt.show()

    # early stopping on val L1 stagnation
    if no_improve >= patience:
        print(f"⏹ Early stopping triggered (no val-L1 improvement for {patience} epochs).")
        break

# end training
print("Training finished. Best val L1:", best_val_l1)
torch.save(gen.state_dict(), "/kaggle/working/generator_final.pth")


📘 1️⃣6️⃣ GAN Inference — Restore Fingerprint

Loads trained generator and performs:

altered → restored

Also displays:
- Altered input  
- GAN output  
- Ground truth real fingerprint  


In [ ]:
# ===============================
# 🔍 Fingerprint Restoration Inference (Exact Pix2Pix Architecture)
# ===============================
import torch, os
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import torch.nn as nn
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# ----------------------------
# 1️⃣ Define UNetGenerator (same as training)
# ----------------------------
class UNetGenerator(nn.Module):
    def __init__(self, in_channels=1, out_channels=1, features=64):
        super().__init__()
        # Encoder
        self.down1 = nn.Sequential(
            nn.Conv2d(in_channels, features, 4, 2, 1, bias=True),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.down2 = nn.Sequential(
            nn.Conv2d(features, features*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features*2),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.down3 = nn.Sequential(
            nn.Conv2d(features*2, features*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features*4),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.down4 = nn.Sequential(
            nn.Conv2d(features*4, features*8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features*8),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.down5 = nn.Sequential(
            nn.Conv2d(features*8, features*8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features*8),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.down6 = nn.Sequential(
            nn.Conv2d(features*8, features*8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features*8),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.down7 = nn.Sequential(
            nn.Conv2d(features*8, features*8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features*8),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.down8 = nn.Sequential(
            nn.Conv2d(features*8, features*8, 4, 2, 1, bias=False),
            nn.ReLU(inplace=True)
        )

        # Decoder (mirrors encoder)
        def up(in_ch, out_ch, dropout=False):
            layers = [nn.ConvTranspose2d(in_ch, out_ch, 4, 2, 1, bias=False),
                      nn.BatchNorm2d(out_ch),
                      nn.ReLU(inplace=True)]
            if dropout:
                layers.append(nn.Dropout(0.5))
            return nn.Sequential(*layers)

        self.up1 = up(features*8, features*8, dropout=True)
        self.up2 = up(features*16, features*8, dropout=True)
        self.up3 = up(features*16, features*8, dropout=True)
        self.up4 = up(features*16, features*8)
        self.up5 = up(features*16, features*4)
        self.up6 = up(features*8, features*2)
        self.up7 = up(features*4, features)
        self.up8 = nn.Sequential(
            nn.ConvTranspose2d(features*2, out_channels, 4, 2, 1, bias=True),
            nn.Tanh()
        )

    def forward(self, x):
        e1 = self.down1(x)
        e2 = self.down2(e1)
        e3 = self.down3(e2)
        e4 = self.down4(e3)
        e5 = self.down5(e4)
        e6 = self.down6(e5)
        e7 = self.down7(e6)
        e8 = self.down8(e7)

        d1 = self.up1(e8); d1 = torch.cat([d1, e7], dim=1)
        d2 = self.up2(d1); d2 = torch.cat([d2, e6], dim=1)
        d3 = self.up3(d2); d3 = torch.cat([d3, e5], dim=1)
        d4 = self.up4(d3); d4 = torch.cat([d4, e4], dim=1)
        d5 = self.up5(d4); d5 = torch.cat([d5, e3], dim=1)
        d6 = self.up6(d5); d6 = torch.cat([d6, e2], dim=1)
        d7 = self.up7(d6); d7 = torch.cat([d7, e1], dim=1)
        out = self.up8(d7)
        return out

# ----------------------------
# 2️⃣ Load Trained Generator
# ----------------------------
gen = UNetGenerator(in_channels=1, out_channels=1).to(device)
checkpoint_path = "/kaggle/input/fingerprints-models/other/default/1/generator_best.pth"
gen.load_state_dict(torch.load(checkpoint_path, map_location=device), strict=True)
gen.eval()
print("✅ Generator loaded successfully (exact training architecture).")

# ----------------------------
# 3️⃣ Transform
# ----------------------------
transform_gan = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# ----------------------------
# 4️⃣ Load Example Fingerprints
# ----------------------------
altered_path = "/kaggle/input/socofing/SOCOFing/Altered/Altered-Easy/100__M_Left_index_finger_CR.BMP"
real_path = "/kaggle/input/socofing/SOCOFing/Real/100__M_Left_index_finger.BMP"

altered = Image.open(altered_path).convert("L")
real = Image.open(real_path).convert("L")

altered_tensor = transform_gan(altered).unsqueeze(0).to(device)

# ----------------------------
# 5️⃣ Inference
# ----------------------------
with torch.no_grad():
    restored = gen(altered_tensor).cpu()[0,0].numpy()

# Denormalize for visualization
restored = (restored * 0.5 + 0.5)
altered = np.array(altered.resize((256,256)), dtype=np.float32) / 255.0
real = np.array(real.resize((256,256)), dtype=np.float32) / 255.0

plt.figure(figsize=(10,4))
plt.subplot(1,3,1); plt.imshow(altered, cmap='gray'); plt.title("Altered Input"); plt.axis('off')
plt.subplot(1,3,2); plt.imshow(restored, cmap='gray'); plt.title("Restored (GAN Output)"); plt.axis('off')
plt.subplot(1,3,3); plt.imshow(real, cmap='gray'); plt.title("Ground Truth (Real)"); plt.axis('off')
plt.suptitle("Fingerprint Restoration: Altered → Restored → Real", fontsize=14)
plt.tight_layout()
plt.show()


📘 1️⃣7️⃣ Quantitative Evaluation

We evaluate restoration quality using:

- PSNR (clarity)  
- SSIM (structural integrity)  
- MSE (error)  

Then display several test comparisons.


In [ ]:
# ===============================
# 🔢 Quantitative Evaluation (PSNR, SSIM, MSE) - Standalone Version
# ===============================
import os, torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms
from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim, mean_squared_error as mse

# Folder paths
altered_root = "/kaggle/input/socofing/SOCOFing/Altered/Altered-Easy"
real_root = "/kaggle/input/socofing/SOCOFing/Real"

# Example subset of test pairs (same finger, different types)
test_samples = [
    "100__M_Left_index_finger_CR.BMP",
    "101__M_Left_index_finger_CR.BMP",
    "102__M_Left_index_finger_CR.BMP",
    "103__M_Left_index_finger_CR.BMP",
    "104__M_Left_index_finger_CR.BMP"
]

transform_eval = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

psnr_scores, ssim_scores, mse_scores = [], [], []

print("Evaluating fingerprint restoration on", len(test_samples), "pairs...")
gen.eval()

for fname in test_samples:
    altered_path = os.path.join(altered_root, fname)
    real_path = os.path.join(real_root, fname.replace("_CR", ""))  # matching real file

    if not os.path.exists(altered_path) or not os.path.exists(real_path):
        print("⚠️ Missing:", fname)
        continue

    altered = Image.open(altered_path).convert("L")
    real = Image.open(real_path).convert("L")

    altered_tensor = transform_eval(altered).unsqueeze(0).to(device)
    with torch.no_grad():
        restored = gen(altered_tensor).cpu()[0,0].numpy()

    # Denormalize and resize
    restored = (restored * 0.5 + 0.5)
    altered_resized = np.array(altered.resize((256,256)), dtype=np.float32) / 255.0
    real_resized = np.array(real.resize((256,256)), dtype=np.float32) / 255.0

    # Convert for metric computation
    f = (restored * 255).astype(np.uint8)
    r = (real_resized * 255).astype(np.uint8)
    psnr_val = psnr(r, f, data_range=255)
    ssim_val = ssim(r, f, data_range=255)
    mse_val = mse(r, f)

    psnr_scores.append(psnr_val)
    ssim_scores.append(ssim_val)
    mse_scores.append(mse_val)

    # Visualization per sample
    plt.figure(figsize=(9,3))
    plt.subplot(1,3,1); plt.imshow(altered_resized, cmap='gray'); plt.title("Altered"); plt.axis('off')
    plt.subplot(1,3,2); plt.imshow(restored, cmap='gray'); plt.title("Restored"); plt.axis('off')
    plt.subplot(1,3,3); plt.imshow(real_resized, cmap='gray'); plt.title("Real"); plt.axis('off')
    plt.suptitle(f"{fname}\nPSNR: {psnr_val:.2f} dB | SSIM: {ssim_val:.4f} | MSE: {mse_val:.6f}", fontsize=10)
    plt.tight_layout()
    plt.show()

# Print overall averages
print("=============================================")
print(f"🔹 Mean PSNR : {np.mean(psnr_scores):.3f} dB")
print(f"🔹 Mean SSIM : {np.mean(ssim_scores):.4f}")
print(f"🔹 Mean MSE  : {np.mean(mse_scores):.6f}")
print("=============================================")


| Metric   | Your Result | Typical Range                        | Interpretation                                                                                            |
| :------- | :---------- | :----------------------------------- | :-------------------------------------------------------------------------------------------------------- |
| **PSNR** | 21.89 dB    | 20–25 (good), >30 (excellent)        | Your generator is producing clear restorations — not perfect, but visually meaningful ridge recovery.     |
| **SSIM** | **0.943**   | >0.85 = good, >0.90 = excellent      | Excellent structural similarity — ridge patterns and textures are well-aligned with the real fingerprint. |
| **MSE**  | 433.62      | Lower is better (depends on scaling) | Very reasonable for normalized 8-bit grayscale images (0–255).                                            |


📘 1️⃣8️⃣ Create Subset for Web App

Creates a compact dataset used in the Colab Web App:

- Random sample from each folder  
- Organized into proper directories  
- Zipped for download  

Output:
SOCOFing_subset.zip


In [ ]:
import os, shutil, random, zipfile, glob

# Original dataset base
base = "/kaggle/input/socofing/SOCOFing"

# Target subset folder
subset_base = "/kaggle/working/SOCOFing_subset/SOCOFing"
os.makedirs(subset_base, exist_ok=True)

# Define subfolders to include
folders = [
    "Altered/Altered-Easy",
    "Altered/Altered-Medium",
    "Altered/Altered-Hard",
    "Real"
]

# Number of files per folder
num_samples = 15  # You can increase or decrease as needed

# Copy small subset
for folder in folders:
    src = os.path.join(base, folder)
    dst = os.path.join(subset_base, folder)
    os.makedirs(dst, exist_ok=True)
    all_files = glob.glob(os.path.join(src, "*.BMP"))
    sample_files = random.sample(all_files, min(num_samples, len(all_files)))
    for file in sample_files:
        shutil.copy(file, dst)
    print(f"✅ Copied {len(sample_files)} files from {folder}")

# Zip the subset
zip_path = "/kaggle/working/SOCOFing_subset.zip"
with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as zipf:
    for root, _, files in os.walk("/kaggle/working/SOCOFing_subset"):
        for file in files:
            full_path = os.path.join(root, file)
            zipf.write(full_path, os.path.relpath(full_path, "/kaggle/working"))

print(f"\n🎯 Subset created and zipped at: {zip_path}")


# Run this in Colab

📘 1️⃣ Notebook Introduction & Setup Instructions

# 🖐️ Fingerprint AI Web App — Classification + Restoration (Colab Version)

This notebook loads the *trained models* generated from the Kaggle training notebook:

- **fingerprint_classifier.pth** (ResNet50)
- **generator_best.pth** (Pix2Pix GAN)
- **generator_final.pth** (fallback)

You will upload these model files to your Google Drive before running the app.

### 🔧 What this Colab Notebook Does
• Loads classification & restoration models  
• Builds a complete Flask web application  
• Provides endpoints for:
   - Fingerprint Classification  
   - Altered Fingerprint Restoration (GAN)  
• Launches public URL using ngrok  

### 📁 Required Folder Structure in Google Drive
Inside your Google Drive, create:



🧩 2️⃣ Install Dependencies + Create Folders

## 🧩 Install Required Libraries & Prepare Workspace

This cell installs all required dependencies for:
- Flask web server
- Torch + torchvision
- Image processing (Pillow, OpenCV)
- ngrok tunneling for public URL

It also creates the folders:
- templates/
- static/
- uploads/

These are needed for the app structure.


In [ ]:
# ===============================
# 1️⃣ Install Dependencies
# ===============================
!pip install -q flask torch torchvision pillow opencv-python-headless pyngrok --upgrade
!mkdir -p templates static uploads
print("✅ Dependencies installed & folders created.")

🔗 3️⃣ Mount Google Drive & Verify Model Files

## 🔗 Mount Google Drive & Confirm Model Availability

Before proceeding, make sure the trained models are already inside your Drive folder:

Fingerprint Classification & Restoration/
  ├── fingerprint_classifier.pth
  ├── generator_best.pth
  └── generator_final.pth

The following code:
- Mounts your Google Drive
- Lists and confirms model file availability


In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

In [ ]:
!ls "/content/drive/My Drive/Colab Notebooks/On-Going Projects/Fingerprints Project-83/"

In [ ]:
!ls "/content/drive/My Drive/Colab Notebooks/On-Going Projects/Fingerprints Project-83/Fingerprint Classification & Restoration/"

🏗️ 4️⃣ Creating the Flask App (app.py)

## 🏗️ Building Flask Backend (app.py)

This cell creates the entire Flask backend including:

- Upload API
- Classification route
- Restoration route
- Model loading functions
- Image preprocessing (matching Kaggle training)
- Saving restored fingerprint outputs

⚠️ VERY IMPORTANT:
Do NOT modify the architecture of:
- ResNet50 classifier
- UNet generator

They must stay identical to Kaggle models.

Paste the following code into app.py using `%%writefile`.


🧠 5️⃣ Load Models (Classifier + GAN Generator)

## 🧠 Loading Pretrained Models

This section loads the models you trained in your Kaggle notebook.

✔ ResNet50 (real vs altered classification)  
✔ Pix2Pix U-Net Generator (altered → restored)  

These paths point to your Google Drive folder.  
If generator_best fails, the notebook automatically loads generator_final.


In [ ]:
# ===============================
# Flask App Setup — Fixed Architecture & Image Processing
# ===============================
%%writefile app.py
import os, uuid, torch, numpy as np
from flask import Flask, render_template, request, redirect, url_for, send_from_directory, flash, session
from werkzeug.utils import secure_filename
from PIL import Image
from torchvision import transforms, models
import torch.nn as nn

app = Flask(__name__)
app.secret_key = "fingerprint_ai_" + str(uuid.uuid4())[:8]
app.config["UPLOAD_FOLDER"] = "uploads"
os.makedirs(app.config["UPLOAD_FOLDER"], exist_ok=True)
app.config["MAX_CONTENT_LENGTH"] = 50 * 1024 * 1024

def get_device():
    return torch.device("cuda" if torch.cuda.is_available() else "cpu")

device = get_device()
print("🔹 Using device:", device)

def read_image(path, mode="RGB"):
    return Image.open(path).convert(mode)

def save_image(tensor, path):
    """Fixed image saving to match Kaggle output"""
    img = tensor.detach().cpu().squeeze(0).numpy()

    # Handle single channel grayscale (from GAN)
    if img.ndim == 3 and img.shape[0] == 1:
        img = img[0]  # Extract the single channel

    # Denormalize from [-1, 1] to [0, 1]
    img = (img * 0.5 + 0.5)
    img = np.clip(img, 0, 1)

    # Convert to uint8 [0, 255]
    img = (img * 255).astype(np.uint8)

    # Save as grayscale
    Image.fromarray(img, mode='L').save(path)

BASE_DIR = "/content/drive/My Drive/Colab Notebooks/On-Going Projects/Fingerprints Project-83/Fingerprint Classification & Restoration"
CLASSIFIER_PATH = os.path.join(BASE_DIR, "fingerprint_classifier.pth")
GEN_BEST_PATH = os.path.join(BASE_DIR, "generator_best.pth")
GEN_FINAL_PATH = os.path.join(BASE_DIR, "generator_final.pth")

# -----------------------------
# 1️⃣ Classifier — ResNet50 (MATCHING KAGGLE)
# -----------------------------
def load_classifier():
    """Load ResNet50 classifier exactly as trained in Kaggle"""
    model = models.resnet50(weights=None)  # No pretrained weights
    model.fc = nn.Linear(model.fc.in_features, 2)  # Binary classification
    model = model.to(device)

    # Load state dict
    state_dict = torch.load(CLASSIFIER_PATH, map_location=device)
    model.load_state_dict(state_dict, strict=True)
    model.eval()
    print("✅ Loaded ResNet50 classifier successfully.")
    return model

# -----------------------------
# 2️⃣ Pix2Pix UNet Generator (exact Kaggle version)
# -----------------------------
class UNetGenerator(nn.Module):
    def __init__(self, in_channels=1, out_channels=1, features=64):
        super().__init__()
        self.down1 = nn.Sequential(
            nn.Conv2d(in_channels, features, 4, 2, 1, bias=True),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.down2 = nn.Sequential(
            nn.Conv2d(features, features*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features*2),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.down3 = nn.Sequential(
            nn.Conv2d(features*2, features*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features*4),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.down4 = nn.Sequential(
            nn.Conv2d(features*4, features*8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features*8),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.down5 = nn.Sequential(
            nn.Conv2d(features*8, features*8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features*8),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.down6 = nn.Sequential(
            nn.Conv2d(features*8, features*8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features*8),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.down7 = nn.Sequential(
            nn.Conv2d(features*8, features*8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features*8),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.down8 = nn.Sequential(
            nn.Conv2d(features*8, features*8, 4, 2, 1, bias=False),
            nn.ReLU(inplace=True)
        )

        def up(in_ch, out_ch, dropout=False):
            layers = [nn.ConvTranspose2d(in_ch, out_ch, 4, 2, 1, bias=False),
                      nn.BatchNorm2d(out_ch),
                      nn.ReLU(inplace=True)]
            if dropout:
                layers.append(nn.Dropout(0.5))
            return nn.Sequential(*layers)

        self.up1 = up(features*8, features*8, dropout=True)
        self.up2 = up(features*16, features*8, dropout=True)
        self.up3 = up(features*16, features*8, dropout=True)
        self.up4 = up(features*16, features*8)
        self.up5 = up(features*16, features*4)
        self.up6 = up(features*8, features*2)
        self.up7 = up(features*4, features)
        self.up8 = nn.Sequential(
            nn.ConvTranspose2d(features*2, out_channels, 4, 2, 1, bias=True),
            nn.Tanh()
        )

    def forward(self, x):
        e1 = self.down1(x)
        e2 = self.down2(e1)
        e3 = self.down3(e2)
        e4 = self.down4(e3)
        e5 = self.down5(e4)
        e6 = self.down6(e5)
        e7 = self.down7(e6)
        e8 = self.down8(e7)
        d1 = self.up1(e8); d1 = torch.cat([d1, e7], dim=1)
        d2 = self.up2(d1); d2 = torch.cat([d2, e6], dim=1)
        d3 = self.up3(d2); d3 = torch.cat([d3, e5], dim=1)
        d4 = self.up4(d3); d4 = torch.cat([d4, e4], dim=1)
        d5 = self.up5(d4); d5 = torch.cat([d5, e3], dim=1)
        d6 = self.up6(d5); d6 = torch.cat([d6, e2], dim=1)
        d7 = self.up7(d6); d7 = torch.cat([d7, e1], dim=1)
        return self.up8(d7)

# -----------------------------
# Load Models
# -----------------------------
def load_models():
    print("📦 Loading models...")

    classifier = load_classifier()

    generator = UNetGenerator(in_channels=1, out_channels=1).to(device)
    try:
        generator.load_state_dict(torch.load(GEN_BEST_PATH, map_location=device), strict=True)
        print("✅ Loaded generator_best.pth")
    except Exception as e:
        print(f"⚠️ Error loading best: {e}")
        generator.load_state_dict(torch.load(GEN_FINAL_PATH, map_location=device), strict=True)
        print("⚠️ Loaded generator_final.pth instead")

    generator.eval()
    return classifier, generator

classifier, generator = load_models()

# ## 🔍 Prediction Functions

# These functions:

# 1. `classify_fingerprint()`
#    - Performs Kaggle-matched preprocessing: resize → normalize → RGB → ResNet50
#    - Returns label + confidence score

# 2. `restore_fingerprint()`
#    - Converts altered fingerprint to grayscale
#    - Normalizes to [-1, 1]
#    - Runs through Pix2Pix generator
#    - Returns restored fingerprint tensor


# -----------------------------
# Prediction Functions (FIXED TRANSFORMS)
# -----------------------------
def classify_fingerprint(image_path):
    """Classification with exact Kaggle preprocessing"""
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    # Load and convert to RGB (as in Kaggle training)
    image = Image.open(image_path).convert("RGB")
    x = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        logits = classifier(x)
        probs = torch.softmax(logits, dim=1)[0].cpu().numpy()

    label = "REAL" if np.argmax(probs) == 0 else "ALTERED"
    conf = float(np.max(probs))
    return label, conf

def restore_fingerprint(image_path):
    """Restoration with exact Kaggle preprocessing"""
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    # Load as grayscale (as in Kaggle)
    image = Image.open(image_path).convert("L")
    x = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        restored = generator(x)

    return restored

# -----------------------------
# Routes
# -----------------------------
@app.route("/")
def home():
    return render_template("index.html")

@app.route("/classify", methods=["GET", "POST"])
def classify_page():
    result = None
    filename = None

    if request.method == "POST":
        f = request.files.get("image")
        if not f or not f.filename:
            flash("Please upload a fingerprint image.", "warning")
            return redirect(url_for("classify_page"))

        # Save uploaded file
        fname = f"{uuid.uuid4().hex}_{secure_filename(f.filename)}"
        path = os.path.join(app.config["UPLOAD_FOLDER"], fname)
        f.save(path)

        # Classify
        label, conf = classify_fingerprint(path)
        result = {"label": label, "confidence": conf}
        filename = fname

        # Store in session for restoration
        session["last_uploaded"] = path
        if label == "ALTERED":
            session["last_altered"] = path

    return render_template("classify.html", result=result, filename=filename)

@app.route("/restore", methods=["GET", "POST"])
def restore_page():
    if request.method == "POST":
        # Handle direct upload for restoration
        f = request.files.get("image")
        if not f or not f.filename:
            flash("Please upload a fingerprint image.", "warning")
            return redirect(url_for("restore_page"))

        fname = f"{uuid.uuid4().hex}_{secure_filename(f.filename)}"
        altered_path = os.path.join(app.config["UPLOAD_FOLDER"], fname)
        f.save(altered_path)
        session["last_altered"] = altered_path
    else:
        # Use the last altered image from classification
        altered_path = session.get("last_altered", None)

    if not altered_path or not os.path.exists(altered_path):
        flash("No altered fingerprint found. Please upload one or classify first.", "warning")
        return render_template("restore.html", altered_fname=None, restored_fname=None)

    # Perform restoration
    restored_tensor = restore_fingerprint(altered_path)
    restored_fname = "restored_" + os.path.basename(altered_path)
    restored_path = os.path.join(app.config["UPLOAD_FOLDER"], restored_fname)
    save_image(restored_tensor, restored_path)

    return render_template("restore.html",
                         altered_fname=os.path.basename(altered_path),
                         restored_fname=os.path.basename(restored_path))

@app.route("/uploads/<path:fname>")
def uploads(fname):
    return send_from_directory(app.config["UPLOAD_FOLDER"], fname)

if __name__ == "__main__":
    print("🚀 Starting Fingerprint AI Web Server...")
    app.run(host="0.0.0.0", port=8000, debug=False)

🌐 7️⃣ Templates (HTML files)

## 🌐 Web App Frontend (HTML Templates)

The following cells create:
- base.html (global layout)
- index.html (homepage)
- classify.html (classification UI)
- restore.html (restoration UI)

Paste each template exactly into its respective file.


In [ ]:
%%writefile templates/base.html
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>{% block title %}Fingerprint AI{% endblock %}</title>
  <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
  <link rel="preconnect" href="https://fonts.googleapis.com">
  <link href="https://fonts.googleapis.com/css2?family=Inter:wght@300;400;500;600;700;800&display=swap" rel="stylesheet">
</head>
<body>
  <!-- Animated Background -->
  <div class="bg-animation">
    <div class="fingerprint-pattern"></div>
    <div class="gradient-orb orb-1"></div>
    <div class="gradient-orb orb-2"></div>
    <div class="gradient-orb orb-3"></div>
  </div>

  <!-- Navigation -->
  <nav class="navbar">
    <div class="nav-container">
      <a href="{{ url_for('home') }}" class="logo">
        <span class="logo-icon">🔬</span>
        <span class="logo-text">Fingerprint<span class="ai">AI</span></span>
      </a>
      <div class="nav-links">
        <a href="{{ url_for('home') }}" class="nav-link {% if request.endpoint == 'home' %}active{% endif %}">
          <span class="nav-icon">🏠</span> Home
        </a>
        <a href="{{ url_for('classify_page') }}" class="nav-link {% if request.endpoint == 'classify_page' %}active{% endif %}">
          <span class="nav-icon">🔍</span> Classify
        </a>
        <a href="{{ url_for('restore_page') }}" class="nav-link {% if request.endpoint == 'restore_page' %}active{% endif %}">
          <span class="nav-icon">✨</span> Restore
        </a>
      </div>
    </div>
  </nav>

  <!-- Flash Messages -->
  {% with messages = get_flashed_messages(with_categories=true) %}
    {% if messages %}
      <div class="flash-container">
        {% for category, message in messages %}
          <div class="flash-message flash-{{ category }}">
            <span class="flash-icon">{% if category == 'warning' %}⚠️{% else %}ℹ️{% endif %}</span>
            <span>{{ message }}</span>
            <button class="flash-close" onclick="this.parentElement.remove()">✕</button>
          </div>
        {% endfor %}
      </div>
    {% endif %}
  {% endwith %}

  <!-- Main Content -->
  <main class="main-container">
    {% block content %}{% endblock %}
  </main>

  <!-- Footer -->
  <footer class="footer">
    <div class="footer-content">
      <p class="footer-text">
        <span class="footer-icon">🧬</span>
        Developed by <strong>Tarun</strong> | Fingerprint Forensics AI
      </p>
      <p class="footer-subtext">Powered by ResNet50 & Pix2Pix GAN</p>
    </div>
  </footer>

  <script>
    // File upload drag & drop enhancement
    document.addEventListener('DOMContentLoaded', function() {
      const uploadZones = document.querySelectorAll('.upload-zone');

      uploadZones.forEach(zone => {
        const input = zone.querySelector('input[type="file"]');
        const preview = zone.querySelector('.upload-preview');

        ['dragenter', 'dragover', 'dragleave', 'drop'].forEach(eventName => {
          zone.addEventListener(eventName, preventDefaults, false);
        });

        function preventDefaults(e) {
          e.preventDefault();
          e.stopPropagation();
        }

        ['dragenter', 'dragover'].forEach(eventName => {
          zone.addEventListener(eventName, () => zone.classList.add('drag-active'), false);
        });

        ['dragleave', 'drop'].forEach(eventName => {
          zone.addEventListener(eventName, () => zone.classList.remove('drag-active'), false);
        });

        if (input) {
          input.addEventListener('change', function(e) {
            if (this.files && this.files[0]) {
              const reader = new FileReader();
              reader.onload = function(e) {
                if (preview) {
                  preview.innerHTML = `<img src="${e.target.result}" alt="Preview" style="max-width: 100%; border-radius: 8px;">`;
                }
              };
              reader.readAsDataURL(this.files[0]);

              // Update upload text
              const uploadText = zone.querySelector('.upload-text');
              if (uploadText) {
                uploadText.textContent = `Selected: ${this.files[0].name}`;
              }
            }
          });
        }
      });

      // Auto-hide flash messages after 5 seconds
      setTimeout(() => {
        document.querySelectorAll('.flash-message').forEach(msg => {
          msg.style.animation = 'slideOut 0.3s ease';
          setTimeout(() => msg.remove(), 300);
        });
      }, 5000);
    });
  </script>
</body>
</html>

In [ ]:
# ===============================
# 3️⃣ Templates — index.html
# ===============================
%%writefile templates/index.html
{% extends "base.html" %}

{% block title %}Home - Fingerprint AI{% endblock %}

{% block content %}
<div class="hero-section">
  <div class="hero-content">
    <div class="hero-badge">
      <span class="badge-dot"></span>
      AI-Powered Forensic Analysis
    </div>

    <h1 class="hero-title">
      Fingerprint Detection &
      <span class="gradient-text">Restoration</span>
    </h1>

    <p class="hero-subtitle">
      Advanced deep learning models for forensic fingerprint analysis.
      Detect alterations with <strong>99.96% accuracy</strong> and restore
      damaged prints using state-of-the-art Pix2Pix GAN technology.
    </p>

    <div class="hero-buttons">
      <a href="{{ url_for('classify_page') }}" class="btn btn-primary">
        <span class="btn-icon">🔍</span>
        Start Classification
        <span class="btn-arrow">→</span>
      </a>
      <a href="{{ url_for('restore_page') }}" class="btn btn-secondary">
        <span class="btn-icon">✨</span>
        Restore Fingerprint
      </a>
    </div>
  </div>

  <div class="hero-visual">
    <div class="fingerprint-showcase">
      <div class="showcase-card card-1">
        <div class="card-icon">🖐️</div>
        <h3>Real Fingerprint</h3>
        <p>Authentic ridge patterns</p>
      </div>
      <div class="showcase-arrow">→</div>
      <div class="showcase-card card-2">
        <div class="card-icon">⚠️</div>
        <h3>Altered Print</h3>
        <p>Detected modifications</p>
      </div>
      <div class="showcase-arrow">→</div>
      <div class="showcase-card card-3">
        <div class="card-icon">✅</div>
        <h3>Restored Print</h3>
        <p>AI reconstruction</p>
      </div>
    </div>
  </div>
</div>

<div class="features-section">
  <h2 class="section-title">
    <span class="title-icon">⚡</span>
    Key Features
  </h2>

  <div class="features-grid">
    <div class="feature-card">
      <div class="feature-icon">🎯</div>
      <h3>High Accuracy</h3>
      <p>ResNet50-based classifier achieving 99.96% test accuracy on altered fingerprint detection</p>
      <div class="feature-stat">99.96%</div>
    </div>

    <div class="feature-card">
      <div class="feature-icon">🧠</div>
      <h3>Deep Learning</h3>
      <p>Powered by advanced neural networks trained on thousands of forensic fingerprint samples</p>
      <div class="feature-stat">50K+</div>
    </div>

    <div class="feature-card">
      <div class="feature-icon">⚡</div>
      <h3>Fast Processing</h3>
      <p>Real-time classification and restoration with GPU acceleration for instant results</p>
      <div class="feature-stat">&lt;3s</div>
    </div>

    <div class="feature-card">
      <div class="feature-icon">🔬</div>
      <h3>Forensic Grade</h3>
      <p>Meets forensic analysis standards with quantitative metrics (PSNR, SSIM, MSE)</p>
      <div class="feature-stat">0.943</div>
    </div>
  </div>
</div>

<div class="workflow-section">
  <h2 class="section-title">
    <span class="title-icon">🔄</span>
    How It Works
  </h2>

  <div class="workflow-steps">
    <div class="workflow-step">
      <div class="step-number">1</div>
      <div class="step-content">
        <h3>Upload Fingerprint</h3>
        <p>Upload your fingerprint image (BMP, PNG, JPG formats supported)</p>
      </div>
    </div>

    <div class="workflow-connector"></div>

    <div class="workflow-step">
      <div class="step-number">2</div>
      <div class="step-content">
        <h3>AI Analysis</h3>
        <p>ResNet50 classifier detects if the print is real or altered</p>
      </div>
    </div>

    <div class="workflow-connector"></div>

    <div class="workflow-step">
      <div class="step-number">3</div>
      <div class="step-content">
        <h3>Restoration (Optional)</h3>
        <p>Pix2Pix GAN reconstructs altered fingerprints to original state</p>
      </div>
    </div>

    <div class="workflow-connector"></div>

    <div class="workflow-step">
      <div class="step-number">4</div>
      <div class="step-content">
        <h3>Download Results</h3>
        <p>View and download classification results and restored images</p>
      </div>
    </div>
  </div>
</div>

<div class="cta-section">
  <div class="cta-content">
    <h2>Ready to Analyze Your Fingerprints?</h2>
    <p>Experience the power of AI-driven forensic analysis</p>
    <a href="{{ url_for('classify_page') }}" class="btn btn-primary btn-large">
      <span class="btn-icon">🚀</span>
      Get Started Now
      <span class="btn-arrow">→</span>
    </a>
  </div>
</div>
{% endblock %}

In [ ]:
# ===============================
# 4️⃣ Templates — classify.html
# ===============================
%%writefile templates/classify.html
{% extends "base.html" %}

{% block title %}Classify - Fingerprint AI{% endblock %}

{% block content %}
<div class="page-container">
  <div class="page-header">
    <div class="page-icon">🔍</div>
    <h1 class="page-title">Fingerprint Classification</h1>
    <p class="page-description">
      Upload a fingerprint image to detect if it's <strong>Real</strong> or <strong>Altered</strong>
    </p>
  </div>

  <div class="content-grid">
    <!-- Upload Section -->
    <div class="upload-section">
      <form method="post" enctype="multipart/form-data" class="upload-form">
        <div class="upload-zone" id="uploadZone">
          <input type="file" name="image" accept="image/*" required id="fileInput">
          <div class="upload-content">
            <div class="upload-icon-large">🖐️</div>
            <h3 class="upload-title">Drop your fingerprint here</h3>
            <p class="upload-text">or click to browse</p>
            <p class="upload-hint">Supports BMP, PNG, JPG formats</p>
          </div>
          <div class="upload-preview"></div>
        </div>

        <button type="submit" class="btn btn-primary btn-block">
          <span class="btn-icon">🔬</span>
          Analyze Fingerprint
          <span class="btn-arrow">→</span>
        </button>
      </form>
    </div>

    <!-- Results Section -->
    {% if result %}
    <div class="results-section">
      <!-- Preview Image -->
      {% if filename %}
      <div class="preview-card">
        <h3 class="card-title">
          <span class="card-icon">📸</span>
          Uploaded Image
        </h3>
        <div class="image-container">
          <img src="{{ url_for('uploads', fname=filename) }}" alt="Fingerprint" class="preview-image">
        </div>
      </div>
      {% endif %}

      <!-- Classification Result -->
      <div class="result-card">
        <h3 class="card-title">
          <span class="card-icon">🎯</span>
          Classification Result
        </h3>

        <div class="result-display">
          <div class="result-badge {{ 'badge-real' if result.label == 'REAL' else 'badge-altered' }}">
            <span class="badge-icon">{{ '✅' if result.label == 'REAL' else '⚠️' }}</span>
            <span class="badge-label">{{ result.label }}</span>
          </div>

          <div class="confidence-meter">
            <div class="confidence-label">
              <span>Confidence</span>
              <span class="confidence-value">{{ (result.confidence * 100)|round(2) }}%</span>
            </div>
            <div class="confidence-bar">
              <div class="confidence-fill {{ 'fill-real' if result.label == 'REAL' else 'fill-altered' }}"
                   style="width: {{ (result.confidence * 100)|round(2) }}%"></div>
            </div>
          </div>

          <div class="result-description">
            {% if result.label == 'REAL' %}
              <p class="desc-icon">✓</p>
              <p class="desc-text">
                This fingerprint appears to be <strong>authentic</strong> with no detected alterations.
                The ridge patterns match expected characteristics of genuine prints.
              </p>
            {% else %}
              <p class="desc-icon">!</p>
              <p class="desc-text">
                This fingerprint shows signs of <strong>alteration</strong>. Detected modifications
                may include scars, burns, abrasions, or other intentional changes.
              </p>
            {% endif %}
          </div>

          {% if result.label == 'ALTERED' %}
          <div class="action-buttons">
            <a href="{{ url_for('restore_page') }}" class="btn btn-success btn-block">
              <span class="btn-icon">✨</span>
              Restore This Fingerprint
              <span class="btn-arrow">→</span>
            </a>
          </div>
          {% endif %}
        </div>
      </div>

      <!-- Technical Details -->
      <div class="details-card">
        <h3 class="card-title">
          <span class="card-icon">📊</span>
          Technical Details
        </h3>
        <div class="details-grid">
          <div class="detail-item">
            <span class="detail-label">Model</span>
            <span class="detail-value">ResNet50</span>
          </div>
          <div class="detail-item">
            <span class="detail-label">Classes</span>
            <span class="detail-value">Real / Altered</span>
          </div>
          <div class="detail-item">
            <span class="detail-label">Accuracy</span>
            <span class="detail-value">99.96%</span>
          </div>
          <div class="detail-item">
            <span class="detail-label">Input Size</span>
            <span class="detail-value">224×224</span>
          </div>
        </div>
      </div>
    </div>
    {% else %}
    <!-- Instructions (shown when no result) -->
    <div class="instructions-card">
      <div class="instruction-icon">💡</div>
      <h3>Instructions</h3>
      <ul class="instruction-list">
        <li><strong>Step 1:</strong> Upload a fingerprint image using the form on the left</li>
        <li><strong>Step 2:</strong> Click "Analyze Fingerprint" to run the classification</li>
        <li><strong>Step 3:</strong> View the results showing if the print is Real or Altered</li>
        <li><strong>Step 4:</strong> If altered, click "Restore" to reconstruct the original print</li>
      </ul>

      <div class="info-box">
        <span class="info-icon">ℹ️</span>
        <p>
          Our AI model has been trained on the SOCOFing dataset with over 50,000 fingerprint samples,
          achieving state-of-the-art accuracy in detecting alterations.
        </p>
      </div>
    </div>
    {% endif %}
  </div>
</div>
{% endblock %}


In [ ]:
# ===============================
# 5️⃣ Templates — restore.html (Auto-display version)
# ===============================
%%writefile templates/restore.html
{% extends "base.html" %}

{% block title %}Restore - Fingerprint AI{% endblock %}

{% block content %}
<div class="page-container">
  <div class="page-header">
    <div class="page-icon">✨</div>
    <h1 class="page-title">Fingerprint Restoration</h1>
    <p class="page-description">
      Reconstruct altered fingerprints using advanced <strong>Pix2Pix GAN</strong> technology
    </p>
  </div>

  {% if not altered_fname and not restored_fname %}
  <!-- Upload Option for Direct Restoration -->
  <div class="content-grid">
    <div class="upload-section">
      <div class="info-banner">
        <span class="banner-icon">💡</span>
        <p>You can either upload a new altered fingerprint here, or classify one first from the Classification page.</p>
      </div>

      <form method="post" enctype="multipart/form-data" class="upload-form">
        <div class="upload-zone" id="uploadZone">
          <input type="file" name="image" accept="image/*" required id="fileInput">
          <div class="upload-content">
            <div class="upload-icon-large">🖐️</div>
            <h3 class="upload-title">Drop altered fingerprint here</h3>
            <p class="upload-text">or click to browse</p>
            <p class="upload-hint">Upload the fingerprint you want to restore</p>
          </div>
          <div class="upload-preview"></div>
        </div>

        <button type="submit" class="btn btn-success btn-block">
          <span class="btn-icon">✨</span>
          Restore Fingerprint
          <span class="btn-arrow">→</span>
        </button>
      </form>
    </div>

    <div class="instructions-card">
      <div class="instruction-icon">🔬</div>
      <h3>How Restoration Works</h3>
      <div class="process-steps">
        <div class="process-step">
          <div class="step-badge">1</div>
          <div class="step-info">
            <h4>Input Analysis</h4>
            <p>The U-Net generator analyzes the altered fingerprint's ridge patterns and identifies modifications</p>
          </div>
        </div>
        <div class="process-step">
          <div class="step-badge">2</div>
          <div class="step-info">
            <h4>Pattern Reconstruction</h4>
            <p>The GAN reconstructs original ridge structures using learned patterns from thousands of authentic prints</p>
          </div>
        </div>
        <div class="process-step">
          <div class="step-badge">3</div>
          <div class="step-info">
            <h4>Quality Enhancement</h4>
            <p>Final output is refined to match forensic standards with high PSNR (~22dB) and SSIM (0.943)</p>
          </div>
        </div>
      </div>

      <div class="metrics-box">
        <h4>Performance Metrics</h4>
        <div class="metrics-grid">
          <div class="metric-item">
            <span class="metric-label">PSNR</span>
            <span class="metric-value">21.89 dB</span>
          </div>
          <div class="metric-item">
            <span class="metric-label">SSIM</span>
            <span class="metric-value">0.943</span>
          </div>
          <div class="metric-item">
            <span class="metric-label">MSE</span>
            <span class="metric-value">433.62</span>
          </div>
        </div>
      </div>
    </div>
  </div>
  {% endif %}

  {% if altered_fname and restored_fname %}
  <!-- Restoration Results -->
  <div class="restoration-results">
    <div class="results-header">
      <div class="success-badge">
        <span class="badge-icon">✅</span>
        <span>Restoration Complete</span>
      </div>
      <p class="results-subtitle">Compare the altered and restored fingerprints below</p>
    </div>

    <div class="comparison-container">
      <!-- Before (Altered) -->
      <div class="comparison-card card-before">
        <div class="card-header">
          <span class="card-icon">⚠️</span>
          <h3>Altered Input</h3>
        </div>
        <div class="image-wrapper">
          <img src="{{ url_for('uploads', fname=altered_fname) }}" alt="Altered Fingerprint" class="comparison-image">
          <div class="image-label label-before">BEFORE</div>
        </div>
        <div class="card-footer">
          <p>Original altered fingerprint with modifications</p>
        </div>
      </div>

      <!-- Comparison Arrow -->
      <div class="comparison-arrow">
        <div class="arrow-icon">→</div>
        <div class="arrow-label">AI Restoration</div>
      </div>

      <!-- After (Restored) -->
      <div class="comparison-card card-after">
        <div class="card-header">
          <span class="card-icon">✨</span>
          <h3>Restored Output</h3>
        </div>
        <div class="image-wrapper">
          <img src="{{ url_for('uploads', fname=restored_fname) }}" alt="Restored Fingerprint" class="comparison-image">
          <div class="image-label label-after">AFTER</div>
        </div>
        <div class="card-footer">
          <p>Reconstructed fingerprint with enhanced ridges</p>
        </div>
      </div>
    </div>

    <!-- Analysis Details -->
    <div class="analysis-section">
      <h3 class="section-subtitle">
        <span class="subtitle-icon">📊</span>
        Restoration Analysis
      </h3>

      <div class="analysis-grid">
        <div class="analysis-card">
          <div class="analysis-icon">🎯</div>
          <h4>Accuracy</h4>
          <p>Ridge patterns reconstructed with high fidelity matching authentic fingerprint structures</p>
        </div>

        <div class="analysis-card">
          <div class="analysis-icon">🔍</div>
          <h4>Quality Metrics</h4>
          <p>SSIM score of 0.943 indicates excellent structural similarity to original patterns</p>
        </div>

        <div class="analysis-card">
          <div class="analysis-icon">⚡</div>
          <h4>Processing Time</h4>
          <p>Restoration completed in under 3 seconds using GPU-accelerated inference</p>
        </div>

        <div class="analysis-card">
          <div class="analysis-icon">🔬</div>
          <h4>Model Architecture</h4>
          <p>Pix2Pix GAN with U-Net generator trained on 10,000+ fingerprint pairs</p>
        </div>
      </div>
    </div>

    <!-- Action Buttons -->
    <div class="action-section">
      <a href="{{ url_for('uploads', fname=restored_fname) }}" download class="btn btn-primary">
        <span class="btn-icon">💾</span>
        Download Restored Image
      </a>
      <a href="{{ url_for('restore_page') }}" class="btn btn-secondary">
        <span class="btn-icon">🔄</span>
        Restore Another
      </a>
      <a href="{{ url_for('classify_page') }}" class="btn btn-outline">
        <span class="btn-icon">🔍</span>
        Classify New Image
      </a>
    </div>

    <!-- Technical Note -->
    <div class="tech-note">
      <span class="note-icon">ℹ️</span>
      <p>
        <strong>Note:</strong> The restored fingerprint is a reconstruction based on learned patterns
        from authentic prints. While highly accurate (PSNR: 21.89 dB), it should be used for forensic
        reference purposes. Always compare with the original altered print for investigation.
      </p>
    </div>
  </div>
  {% endif %}
</div>
{% endblock %}


🎨 8️⃣ CSS Stylesheet

## 🎨 Modern UI Styling

This stylesheet contains:
- Dark theme
- Gradients
- Animations
- Responsiveness
- Card layouts for classification & restoration

Place it inside static/style.css


In [ ]:
# ===============================
# 6️⃣ CSS — static/style.css
# ===============================
%%writefile static/style.css
/* ==========================================
   FINGERPRINT AI - MODERN UI STYLES
   ========================================== */

/* CSS Variables */
:root {
  /* Color Palette */
  --bg-primary: #0a0e27;
  --bg-secondary: #131842;
  --bg-card: rgba(255, 255, 255, 0.03);
  --bg-card-hover: rgba(255, 255, 255, 0.06);

  /* Gradients */
  --gradient-primary: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
  --gradient-success: linear-gradient(135deg, #11998e 0%, #38ef7d 100%);
  --gradient-warning: linear-gradient(135deg, #f093fb 0%, #f5576c 100%);
  --gradient-info: linear-gradient(135deg, #4facfe 0%, #00f2fe 100%);

  /* Text Colors */
  --text-primary: #ffffff;
  --text-secondary: #a0aec0;
  --text-muted: #718096;

  /* Accent Colors */
  --accent-primary: #667eea;
  --accent-success: #38ef7d;
  --accent-warning: #f5576c;
  --accent-info: #00f2fe;

  /* Borders */
  --border-color: rgba(255, 255, 255, 0.1);
  --border-hover: rgba(255, 255, 255, 0.2);

  /* Shadows */
  --shadow-sm: 0 2px 8px rgba(0, 0, 0, 0.1);
  --shadow-md: 0 4px 16px rgba(0, 0, 0, 0.2);
  --shadow-lg: 0 10px 40px rgba(0, 0, 0, 0.3);
  --shadow-xl: 0 20px 60px rgba(0, 0, 0, 0.4);

  /* Spacing */
  --spacing-xs: 0.5rem;
  --spacing-sm: 1rem;
  --spacing-md: 1.5rem;
  --spacing-lg: 2rem;
  --spacing-xl: 3rem;

  /* Borders & Radius */
  --radius-sm: 8px;
  --radius-md: 12px;
  --radius-lg: 16px;
  --radius-xl: 24px;

  /* Transitions */
  --transition-fast: 0.2s ease;
  --transition-base: 0.3s ease;
  --transition-slow: 0.5s ease;
}

/* ==========================================
   GLOBAL STYLES
   ========================================== */

* {
  margin: 0;
  padding: 0;
  box-sizing: border-box;
}

html {
  scroll-behavior: smooth;
}

body {
  font-family: 'Inter', -apple-system, BlinkMacSystemFont, 'Segoe UI', sans-serif;
  background: var(--bg-primary);
  color: var(--text-primary);
  line-height: 1.6;
  min-height: 100vh;
  overflow-x: hidden;
  position: relative;
}

/* ==========================================
   ANIMATED BACKGROUND
   ========================================== */

.bg-animation {
  position: fixed;
  top: 0;
  left: 0;
  width: 100%;
  height: 100%;
  z-index: 0;
  overflow: hidden;
  pointer-events: none;
}

.fingerprint-pattern {
  position: absolute;
  width: 100%;
  height: 100%;
  background-image:
    repeating-linear-gradient(90deg, rgba(102, 126, 234, 0.03) 0px, transparent 1px, transparent 40px),
    repeating-linear-gradient(0deg, rgba(102, 126, 234, 0.03) 0px, transparent 1px, transparent 40px);
  opacity: 0.5;
}

.gradient-orb {
  position: absolute;
  border-radius: 50%;
  filter: blur(80px);
  opacity: 0.3;
  animation: float 20s ease-in-out infinite;
}

.orb-1 {
  width: 400px;
  height: 400px;
  background: radial-gradient(circle, #667eea, transparent);
  top: -200px;
  left: -200px;
  animation-delay: 0s;
}

.orb-2 {
  width: 500px;
  height: 500px;
  background: radial-gradient(circle, #764ba2, transparent);
  bottom: -250px;
  right: -250px;
  animation-delay: 5s;
}

.orb-3 {
  width: 350px;
  height: 350px;
  background: radial-gradient(circle, #00f2fe, transparent);
  top: 50%;
  left: 50%;
  animation-delay: 10s;
}

@keyframes float {
  0%, 100% { transform: translate(0, 0) scale(1); }
  33% { transform: translate(50px, -50px) scale(1.1); }
  66% { transform: translate(-50px, 50px) scale(0.9); }
}

/* ==========================================
   NAVIGATION BAR
   ========================================== */

.navbar {
  position: sticky;
  top: 0;
  z-index: 1000;
  background: rgba(10, 14, 39, 0.8);
  backdrop-filter: blur(20px);
  border-bottom: 1px solid var(--border-color);
  padding: var(--spacing-sm) 0;
}

.nav-container {
  max-width: 1400px;
  margin: 0 auto;
  padding: 0 var(--spacing-lg);
  display: flex;
  justify-content: space-between;
  align-items: center;
}

.logo {
  display: flex;
  align-items: center;
  gap: var(--spacing-xs);
  text-decoration: none;
  font-size: 1.5rem;
  font-weight: 700;
  color: var(--text-primary);
  transition: var(--transition-base);
}

.logo-icon {
  font-size: 2rem;
  animation: pulse 2s ease-in-out infinite;
}

.logo-text {
  background: var(--gradient-primary);
  -webkit-background-clip: text;
  -webkit-text-fill-color: transparent;
  background-clip: text;
}

.logo .ai {
  color: var(--accent-info);
}

.logo:hover {
  transform: translateY(-2px);
}

@keyframes pulse {
  0%, 100% { transform: scale(1); }
  50% { transform: scale(1.1); }
}

.nav-links {
  display: flex;
  gap: var(--spacing-sm);
}

.nav-link {
  display: flex;
  align-items: center;
  gap: 0.5rem;
  padding: 0.75rem 1.25rem;
  text-decoration: none;
  color: var(--text-secondary);
  font-weight: 500;
  border-radius: var(--radius-md);
  transition: var(--transition-base);
  position: relative;
}

.nav-link::before {
  content: '';
  position: absolute;
  bottom: 0;
  left: 50%;
  width: 0;
  height: 2px;
  background: var(--gradient-primary);
  transform: translateX(-50%);
  transition: var(--transition-base);
}

.nav-link:hover {
  color: var(--text-primary);
  background: var(--bg-card);
}

.nav-link:hover::before,
.nav-link.active::before {
  width: 80%;
}

.nav-link.active {
  color: var(--text-primary);
  background: var(--bg-card);
}

.nav-icon {
  font-size: 1.2rem;
}

/* ==========================================
   FLASH MESSAGES
   ========================================== */

.flash-container {
  position: fixed;
  top: 80px;
  right: var(--spacing-lg);
  z-index: 2000;
  max-width: 400px;
  animation: slideIn 0.3s ease;
}

.flash-message {
  display: flex;
  align-items: center;
  gap: var(--spacing-sm);
  padding: var(--spacing-md);
  background: var(--bg-secondary);
  border: 1px solid var(--border-color);
  border-radius: var(--radius-md);
  box-shadow: var(--shadow-lg);
  margin-bottom: var(--spacing-sm);
  animation: slideIn 0.3s ease;
}

.flash-warning {
  border-left: 4px solid var(--accent-warning);
}

.flash-icon {
  font-size: 1.5rem;
  flex-shrink: 0;
}

.flash-close {
  margin-left: auto;
  background: none;
  border: none;
  color: var(--text-secondary);
  cursor: pointer;
  font-size: 1.2rem;
  padding: 0.25rem 0.5rem;
  border-radius: var(--radius-sm);
  transition: var(--transition-fast);
}

.flash-close:hover {
  background: var(--bg-card);
  color: var(--text-primary);
}

@keyframes slideIn {
  from {
    transform: translateX(100%);
    opacity: 0;
  }
  to {
    transform: translateX(0);
    opacity: 1;
  }
}

@keyframes slideOut {
  from {
    transform: translateX(0);
    opacity: 1;
  }
  to {
    transform: translateX(100%);
    opacity: 0;
  }
}

/* ==========================================
   MAIN CONTAINER
   ========================================== */

.main-container {
  position: relative;
  z-index: 1;
  max-width: 1400px;
  margin: 0 auto;
  padding: var(--spacing-xl) var(--spacing-lg);
  min-height: calc(100vh - 180px);
}

/* ==========================================
   HERO SECTION (Homepage)
   ========================================== */

.hero-section {
  display: grid;
  grid-template-columns: 1fr 1fr;
  gap: var(--spacing-xl);
  align-items: center;
  padding: var(--spacing-xl) 0;
  min-height: 70vh;
}

.hero-content {
  animation: fadeInLeft 0.8s ease;
}

.hero-badge {
  display: inline-flex;
  align-items: center;
  gap: var(--spacing-xs);
  padding: 0.5rem 1rem;
  background: var(--bg-card);
  border: 1px solid var(--border-color);
  border-radius: 50px;
  font-size: 0.875rem;
  font-weight: 500;
  color: var(--text-secondary);
  margin-bottom: var(--spacing-md);
}

.badge-dot {
  width: 8px;
  height: 8px;
  background: var(--accent-success);
  border-radius: 50%;
  animation: blink 2s ease-in-out infinite;
}

@keyframes blink {
  0%, 100% { opacity: 1; }
  50% { opacity: 0.3; }
}

.hero-title {
  font-size: 3.5rem;
  font-weight: 800;
  line-height: 1.2;
  margin-bottom: var(--spacing-md);
  color: var(--text-primary);
}

.gradient-text {
  background: var(--gradient-primary);
  -webkit-background-clip: text;
  -webkit-text-fill-color: transparent;
  background-clip: text;
}

.hero-subtitle {
  font-size: 1.25rem;
  color: var(--text-secondary);
  line-height: 1.8;
  margin-bottom: var(--spacing-lg);
}

.hero-buttons {
  display: flex;
  gap: var(--spacing-md);
  flex-wrap: wrap;
}

/* Hero Visual */
.hero-visual {
  animation: fadeInRight 0.8s ease;
}

.fingerprint-showcase {
  display: flex;
  align-items: center;
  justify-content: center;
  gap: var(--spacing-md);
  flex-wrap: wrap;
}

.showcase-card {
  background: var(--bg-card);
  border: 1px solid var(--border-color);
  border-radius: var(--radius-lg);
  padding: var(--spacing-lg);
  text-align: center;
  min-width: 150px;
  transition: var(--transition-base);
  animation: float 3s ease-in-out infinite;
}

.card-1 { animation-delay: 0s; }
.card-2 { animation-delay: 0.5s; }
.card-3 { animation-delay: 1s; }

.showcase-card:hover {
  background: var(--bg-card-hover);
  border-color: var(--border-hover);
  transform: translateY(-5px);
}

.card-icon {
  font-size: 3rem;
  margin-bottom: var(--spacing-sm);
}

.showcase-card h3 {
  font-size: 1rem;
  margin-bottom: 0.5rem;
  color: var(--text-primary);
}

.showcase-card p {
  font-size: 0.875rem;
  color: var(--text-secondary);
}

.showcase-arrow {
  font-size: 2rem;
  color: var(--accent-primary);
  animation: bounce 2s ease-in-out infinite;
}

@keyframes bounce {
  0%, 100% { transform: translateX(0); }
  50% { transform: translateX(10px); }
}

/* ==========================================
   FEATURES SECTION
   ========================================== */

.features-section,
.workflow-section {
  margin: var(--spacing-xl) 0;
  padding: var(--spacing-xl) 0;
}

.section-title {
  display: flex;
  align-items: center;
  justify-content: center;
  gap: var(--spacing-sm);
  font-size: 2.5rem;
  font-weight: 700;
  margin-bottom: var(--spacing-xl);
  text-align: center;
}

.title-icon {
  font-size: 2.5rem;
}

.features-grid {
  display: grid;
  grid-template-columns: repeat(auto-fit, minmax(280px, 1fr));
  gap: var(--spacing-lg);
}

.feature-card {
  background: var(--bg-card);
  border: 1px solid var(--border-color);
  border-radius: var(--radius-lg);
  padding: var(--spacing-lg);
  text-align: center;
  transition: var(--transition-base);
  position: relative;
  overflow: hidden;
}

.feature-card::before {
  content: '';
  position: absolute;
  top: 0;
  left: 0;
  width: 100%;
  height: 4px;
  background: var(--gradient-primary);
  transform: scaleX(0);
  transition: var(--transition-base);
}

.feature-card:hover::before {
  transform: scaleX(1);
}

.feature-card:hover {
  background: var(--bg-card-hover);
  border-color: var(--border-hover);
  transform: translateY(-5px);
  box-shadow: var(--shadow-lg);
}

.feature-icon {
  font-size: 3rem;
  margin-bottom: var(--spacing-md);
  display: inline-block;
  animation: bounce 2s ease-in-out infinite;
}

.feature-card:nth-child(1) .feature-icon { animation-delay: 0s; }
.feature-card:nth-child(2) .feature-icon { animation-delay: 0.2s; }
.feature-card:nth-child(3) .feature-icon { animation-delay: 0.4s; }
.feature-card:nth-child(4) .feature-icon { animation-delay: 0.6s; }

.feature-card h3 {
  font-size: 1.5rem;
  margin-bottom: var(--spacing-sm);
  color: var(--text-primary);
}

.feature-card p {
  color: var(--text-secondary);
  line-height: 1.6;
  margin-bottom: var(--spacing-md);
}

.feature-stat {
  display: inline-block;
  padding: 0.5rem 1rem;
  background: var(--gradient-primary);
  border-radius: 50px;
  font-weight: 700;
  font-size: 1.125rem;
  color: white;
}

/* ==========================================
   WORKFLOW SECTION
   ========================================== */

.workflow-steps {
  display: flex;
  align-items: flex-start;
  justify-content: center;
  gap: var(--spacing-sm);
  flex-wrap: wrap;
}

.workflow-step {
  flex: 1;
  min-width: 200px;
  max-width: 250px;
  background: var(--bg-card);
  border: 1px solid var(--border-color);
  border-radius: var(--radius-lg);
  padding: var(--spacing-lg);
  text-align: center;
  transition: var(--transition-base);
}

.workflow-step:hover {
  background: var(--bg-card-hover);
  transform: translateY(-5px);
  box-shadow: var(--shadow-md);
}

.step-number {
  width: 60px;
  height: 60px;
  margin: 0 auto var(--spacing-md);
  display: flex;
  align-items: center;
  justify-content: center;
  background: var(--gradient-primary);
  border-radius: 50%;
  font-size: 1.5rem;
  font-weight: 700;
  color: white;
}

.step-content h3 {
  font-size: 1.125rem;
  margin-bottom: var(--spacing-sm);
  color: var(--text-primary);
}

.step-content p {
  font-size: 0.875rem;
  color: var(--text-secondary);
  line-height: 1.6;
}

.workflow-connector {
  width: 50px;
  height: 2px;
  background: linear-gradient(90deg, var(--accent-primary), transparent);
  align-self: center;
  margin-top: 40px;
}

/* ==========================================
   CTA SECTION
   ========================================== */

.cta-section {
  background: var(--gradient-primary);
  border-radius: var(--radius-xl);
  padding: var(--spacing-xl);
  text-align: center;
  margin: var(--spacing-xl) 0;
}

.cta-content h2 {
  font-size: 2.5rem;
  margin-bottom: var(--spacing-md);
  color: white;
}

.cta-content p {
  font-size: 1.25rem;
  margin-bottom: var(--spacing-lg);
  color: rgba(255, 255, 255, 0.9);
}

/* ==========================================
   BUTTONS
   ========================================== */

.btn {
  display: inline-flex;
  align-items: center;
  gap: var(--spacing-xs);
  padding: 0.875rem 1.75rem;
  font-size: 1rem;
  font-weight: 600;
  text-decoration: none;
  border: none;
  border-radius: var(--radius-md);
  cursor: pointer;
  transition: var(--transition-base);
  position: relative;
  overflow: hidden;
}

.btn::before {
  content: '';
  position: absolute;
  top: 50%;
  left: 50%;
  width: 0;
  height: 0;
  background: rgba(255, 255, 255, 0.2);
  border-radius: 50%;
  transform: translate(-50%, -50%);
  transition: width 0.6s, height 0.6s;
}

.btn:hover::before {
  width: 300px;
  height: 300px;
}

.btn-primary {
  background: var(--gradient-primary);
  color: white;
  box-shadow: 0 4px 15px rgba(102, 126, 234, 0.4);
}

.btn-primary:hover {
  transform: translateY(-2px);
  box-shadow: 0 6px 20px rgba(102, 126, 234, 0.6);
}

.btn-secondary {
  background: var(--gradient-info);
  color: white;
  box-shadow: 0 4px 15px rgba(0, 242, 254, 0.4);
}

.btn-secondary:hover {
  transform: translateY(-2px);
  box-shadow: 0 6px 20px rgba(0, 242, 254, 0.6);
}

.btn-success {
  background: var(--gradient-success);
  color: white;
  box-shadow: 0 4px 15px rgba(56, 239, 125, 0.4);
}

.btn-success:hover {
  transform: translateY(-2px);
  box-shadow: 0 6px 20px rgba(56, 239, 125, 0.6);
}

.btn-outline {
  background: transparent;
  color: var(--text-primary);
  border: 2px solid var(--border-color);
}

.btn-outline:hover {
  border-color: var(--accent-primary);
  background: var(--bg-card);
}

.btn-large {
  padding: 1.125rem 2.5rem;
  font-size: 1.125rem;
}

.btn-block {
  width: 100%;
  justify-content: center;
}

.btn-icon {
  font-size: 1.25rem;
}

.btn-arrow {
  transition: var(--transition-base);
}

.btn:hover .btn-arrow {
  transform: translateX(5px);
}

/* ==========================================
   PAGE CONTAINER
   ========================================== */

.page-container {
  animation: fadeInUp 0.6s ease;
}

.page-header {
  text-align: center;
  margin-bottom: var(--spacing-xl);
}

.page-icon {
  font-size: 4rem;
  margin-bottom: var(--spacing-md);
  display: inline-block;
  animation: bounce 2s ease-in-out infinite;
}

.page-title {
  font-size: 3rem;
  font-weight: 700;
  margin-bottom: var(--spacing-sm);
  background: var(--gradient-primary);
  -webkit-background-clip: text;
  -webkit-text-fill-color: transparent;
  background-clip: text;
}

.page-description {
  font-size: 1.25rem;
  color: var(--text-secondary);
  max-width: 700px;
  margin: 0 auto;
}

.content-grid {
  display: grid;
  grid-template-columns: 1fr 1fr;
  gap: var(--spacing-xl);
  align-items: start;
}

/* ==========================================
   UPLOAD SECTION
   ========================================== */

.upload-section {
  background: var(--bg-card);
  border: 1px solid var(--border-color);
  border-radius: var(--radius-lg);
  padding: var(--spacing-lg);
}

.upload-form {
  display: flex;
  flex-direction: column;
  gap: var(--spacing-md);
}

.upload-zone {
  position: relative;
  border: 2px dashed var(--border-color);
  border-radius: var(--radius-lg);
  padding: var(--spacing-xl);
  text-align: center;
  background: var(--bg-secondary);
  cursor: pointer;
  transition: var(--transition-base);
  min-height: 300px;
  display: flex;
  align-items: center;
  justify-content: center;
}

.upload-zone:hover,
.upload-zone.drag-active {
  border-color: var(--accent-primary);
  background: rgba(102, 126, 234, 0.05);
}

.upload-zone input[type="file"] {
  position: absolute;
  width: 100%;
  height: 100%;
  top: 0;
  left: 0;
  opacity: 0;
  cursor: pointer;
}

.upload-content {
  width: 100%;
}

.upload-icon-large {
  font-size: 5rem;
  margin-bottom: var(--spacing-md);
  animation: pulse 2s ease-in-out infinite;
}

.upload-title {
  font-size: 1.5rem;
  font-weight: 600;
  margin-bottom: var(--spacing-sm);
  color: var(--text-primary);
}

.upload-text {
  font-size: 1rem;
  color: var(--text-secondary);
  margin-bottom: var(--spacing-xs);
}

.upload-hint {
  font-size: 0.875rem;
  color: var(--text-muted);
}

.upload-preview {
  margin-top: var(--spacing-md);
}

.info-banner {
  display: flex;
  align-items: center;
  gap: var(--spacing-sm);
  padding: var(--spacing-md);
  background: rgba(0, 242, 254, 0.1);
  border: 1px solid rgba(0, 242, 254, 0.3);
  border-radius: var(--radius-md);
  margin-bottom: var(--spacing-md);
}

.banner-icon {
  font-size: 1.5rem;
  flex-shrink: 0;
}

/* ==========================================
   RESULTS SECTION
   ========================================== */

.results-section {
  display: flex;
  flex-direction: column;
  gap: var(--spacing-lg);
}

.preview-card,
.result-card,
.details-card,
.instructions-card {
  background: var(--bg-card);
  border: 1px solid var(--border-color);
  border-radius: var(--radius-lg);
  padding: var(--spacing-lg);
  animation: fadeIn 0.5s ease;
}

.card-title {
  display: flex;
  align-items: center;
  gap: var(--spacing-sm);
  font-size: 1.25rem;
  font-weight: 600;
  margin-bottom: var(--spacing-md);
  color: var(--text-primary);
}

.card-icon {
  font-size: 1.5rem;
}

.image-container {
  border-radius: var(--radius-md);
  overflow: hidden;
  border: 1px solid var(--border-color);
}

.preview-image {
  width: 100%;
  height: auto;
  display: block;
  transition: var(--transition-base);
}

.preview-image:hover {
  transform: scale(1.02);
}

/* Result Display */
.result-display {
  display: flex;
  flex-direction: column;
  gap: var(--spacing-lg);
}

.result-badge {
  display: inline-flex;
  align-items: center;
  gap: var(--spacing-sm);
  padding: var(--spacing-md) var(--spacing-lg);
  border-radius: var(--radius-md);
  font-size: 1.5rem;
  font-weight: 700;
  text-align: center;
  justify-content: center;
  animation: scaleIn 0.5s ease;
}

.badge-real {
  background: rgba(56, 239, 125, 0.2);
  border: 2px solid var(--accent-success);
  color: var(--accent-success);
}

.badge-altered {
  background: rgba(245, 87, 108, 0.2);
  border: 2px solid var(--accent-warning);
  color: var(--accent-warning);
}

.badge-icon {
  font-size: 2rem;
}

.badge-label {
  font-weight: 700;
}

@keyframes scaleIn {
  from {
    transform: scale(0.8);
    opacity: 0;
  }
  to {
    transform: scale(1);
    opacity: 1;
  }
}

/* Confidence Meter */
.confidence-meter {
  background: var(--bg-secondary);
  border-radius: var(--radius-md);
  padding: var(--spacing-md);
}

.confidence-label {
  display: flex;
  justify-content: space-between;
  align-items: center;
  margin-bottom: var(--spacing-sm);
  font-weight: 600;
}

.confidence-value {
  font-size: 1.125rem;
  color: var(--accent-primary);
}

.confidence-bar {
  height: 12px;
  background: rgba(255, 255, 255, 0.1);
  border-radius: 50px;
  overflow: hidden;
}

.confidence-fill {
  height: 100%;
  border-radius: 50px;
  transition: width 1s ease;
  animation: fillBar 1s ease;
}

.fill-real {
  background: var(--gradient-success);
}

.fill-altered {
  background: var(--gradient-warning);
}

@keyframes fillBar {
  from { width: 0; }
}

/* Result Description */
.result-description {
  background: var(--bg-secondary);
  border-radius: var(--radius-md);
  padding: var(--spacing-md);
  display: flex;
  gap: var(--spacing-md);
}

.desc-icon {
  font-size: 2rem;
  flex-shrink: 0;
}

.desc-text {
  color: var(--text-secondary);
  line-height: 1.8;
}

.action-buttons {
  display: flex;
  gap: var(--spacing-md);
}

/* Details Card */
.details-grid {
  display: grid;
  grid-template-columns: repeat(2, 1fr);
  gap: var(--spacing-md);
}

.detail-item {
  display: flex;
  flex-direction: column;
  gap: 0.25rem;
  padding: var(--spacing-md);
  background: var(--bg-secondary);
  border-radius: var(--radius-md);
}

.detail-label {
  font-size: 0.875rem;
  color: var(--text-muted);
  font-weight: 500;
}

.detail-value {
  font-size: 1.125rem;
  color: var(--text-primary);
  font-weight: 600;
}

/* Instructions Card */
.instructions-card {
  text-align: center;
}

.instruction-icon {
  font-size: 4rem;
  margin-bottom: var(--spacing-md);
  display: inline-block;
  animation: pulse 2s ease-in-out infinite;
}

.instructions-card h3 {
  font-size: 1.5rem;
  margin-bottom: var(--spacing-md);
  color: var(--text-primary);
}

.instruction-list {
  list-style: none;
  text-align: left;
  max-width: 600px;
  margin: 0 auto var(--spacing-lg);
}

.instruction-list li {
  padding: var(--spacing-sm);
  margin-bottom: var(--spacing-sm);
  color: var(--text-secondary);
  line-height: 1.8;
  border-left: 3px solid var(--accent-primary);
  padding-left: var(--spacing-md);
}

.info-box {
  display: flex;
  align-items: flex-start;
  gap: var(--spacing-md);
  padding: var(--spacing-md);
  background: rgba(102, 126, 234, 0.1);
  border: 1px solid rgba(102, 126, 234, 0.3);
  border-radius: var(--radius-md);
  text-align: left;
}

.info-icon {
  font-size: 1.5rem;
  flex-shrink: 0;
}

.info-box p {
  color: var(--text-secondary);
  line-height: 1.8;
  margin: 0;
}

/* ==========================================
   RESTORATION PAGE
   ========================================== */

.restoration-results {
  animation: fadeIn 0.6s ease;
}

.results-header {
  text-align: center;
  margin-bottom: var(--spacing-xl);
}

.success-badge {
  display: inline-flex;
  align-items: center;
  gap: var(--spacing-sm);
  padding: var(--spacing-md) var(--spacing-lg);
  background: rgba(56, 239, 125, 0.2);
  border: 2px solid var(--accent-success);
  border-radius: 50px;
  font-size: 1.25rem;
  font-weight: 600;
  color: var(--accent-success);
  margin-bottom: var(--spacing-md);
}

.results-subtitle {
  font-size: 1.125rem;
  color: var(--text-secondary);
}

/* Comparison Container */
.comparison-container {
  display: grid;
  grid-template-columns: 1fr auto 1fr;
  gap: var(--spacing-lg);
  align-items: center;
  margin-bottom: var(--spacing-xl);
}

.comparison-card {
  background: var(--bg-card);
  border: 1px solid var(--border-color);
  border-radius: var(--radius-lg);
  padding: var(--spacing-lg);
  transition: var(--transition-base);
}

.comparison-card:hover {
  background: var(--bg-card-hover);
  border-color: var(--border-hover);
  transform: translateY(-5px);
  box-shadow: var(--shadow-lg);
}

.card-before {
  animation: slideInLeft 0.6s ease;
}

.card-after {
  animation: slideInRight 0.6s ease;
}

.card-header {
  display: flex;
  align-items: center;
  gap: var(--spacing-sm);
  margin-bottom: var(--spacing-md);
}

.card-header h3 {
  font-size: 1.25rem;
  font-weight: 600;
  color: var(--text-primary);
}

.image-wrapper {
  position: relative;
  border-radius: var(--radius-md);
  overflow: hidden;
  border: 1px solid var(--border-color);
  margin-bottom: var(--spacing-md);
}

.comparison-image {
  width: 100%;
  height: auto;
  display: block;
  transition: var(--transition-base);
}

.comparison-image:hover {
  transform: scale(1.05);
}

.image-label {
  position: absolute;
  top: var(--spacing-sm);
  right: var(--spacing-sm);
  padding: 0.5rem 1rem;
  border-radius: 50px;
  font-size: 0.75rem;
  font-weight: 700;
  text-transform: uppercase;
  letter-spacing: 1px;
}

.label-before {
  background: rgba(245, 87, 108, 0.9);
  color: white;
}

.label-after {
  background: rgba(56, 239, 125, 0.9);
  color: white;
}

.card-footer {
  text-align: center;
}

.card-footer p {
  color: var(--text-secondary);
  font-size: 0.875rem;
}

/* Comparison Arrow */
.comparison-arrow {
  display: flex;
  flex-direction: column;
  align-items: center;
  gap: var(--spacing-sm);
}

.arrow-icon {
  font-size: 3rem;
  color: var(--accent-primary);
  animation: pulse 2s ease-in-out infinite;
}

.arrow-label {
  font-size: 0.875rem;
  font-weight: 600;
  color: var(--text-secondary);
  text-transform: uppercase;
  letter-spacing: 1px;
}

/* Analysis Section */
.analysis-section {
  margin: var(--spacing-xl) 0;
}

.section-subtitle {
  display: flex;
  align-items: center;
  gap: var(--spacing-sm);
  font-size: 1.75rem;
  font-weight: 600;
  margin-bottom: var(--spacing-lg);
  color: var(--text-primary);
}

.subtitle-icon {
  font-size: 2rem;
}

.analysis-grid {
  display: grid;
  grid-template-columns: repeat(auto-fit, minmax(250px, 1fr));
  gap: var(--spacing-lg);
}

.analysis-card {
  background: var(--bg-card);
  border: 1px solid var(--border-color);
  border-radius: var(--radius-lg);
  padding: var(--spacing-lg);
  text-align: center;
  transition: var(--transition-base);
}

.analysis-card:hover {
  background: var(--bg-card-hover);
  border-color: var(--border-hover);
  transform: translateY(-5px);
  box-shadow: var(--shadow-md);
}

.analysis-icon {
  font-size: 2.5rem;
  margin-bottom: var(--spacing-md);
  display: inline-block;
}

.analysis-card h4 {
  font-size: 1.125rem;
  margin-bottom: var(--spacing-sm);
  color: var(--text-primary);
}

.analysis-card p {
  color: var(--text-secondary);
  line-height: 1.6;
  font-size: 0.875rem;
}

/* Action Section */
.action-section {
  display: flex;
  gap: var(--spacing-md);
  justify-content: center;
  flex-wrap: wrap;
  margin: var(--spacing-xl) 0;
}

/* Technical Note */
.tech-note {
  display: flex;
  align-items: flex-start;
  gap: var(--spacing-md);
  padding: var(--spacing-lg);
  background: var(--bg-card);
  border: 1px solid var(--border-color);
  border-left: 4px solid var(--accent-info);
  border-radius: var(--radius-md);
}

.note-icon {
  font-size: 1.5rem;
  flex-shrink: 0;
}

.tech-note p {
  color: var(--text-secondary);
  line-height: 1.8;
  margin: 0;
}

/* Process Steps */
.process-steps {
  display: flex;
  flex-direction: column;
  gap: var(--spacing-md);
  margin-bottom: var(--spacing-lg);
}

.process-step {
  display: flex;
  gap: var(--spacing-md);
  align-items: flex-start;
  padding: var(--spacing-md);
  background: var(--bg-secondary);
  border-radius: var(--radius-md);
  transition: var(--transition-base);
}

.process-step:hover {
  background: var(--bg-card-hover);
  transform: translateX(5px);
}

.step-badge {
  width: 40px;
  height: 40px;
  flex-shrink: 0;
  display: flex;
  align-items: center;
  justify-content: center;
  background: var(--gradient-primary);
  border-radius: 50%;
  font-weight: 700;
  color: white;
}

.step-info h4 {
  font-size: 1rem;
  margin-bottom: 0.5rem;
  color: var(--text-primary);
}

.step-info p {
  font-size: 0.875rem;
  color: var(--text-secondary);
  line-height: 1.6;
}

/* Metrics Box */
.metrics-box {
  padding: var(--spacing-md);
  background: var(--bg-secondary);
  border-radius: var(--radius-md);
  margin-top: var(--spacing-md);
}

.metrics-box h4 {
  font-size: 1rem;
  margin-bottom: var(--spacing-md);
  color: var(--text-primary);
  text-align: center;
}

.metrics-grid {
  display: grid;
  grid-template-columns: repeat(3, 1fr);
  gap: var(--spacing-md);
}

.metric-item {
  display: flex;
  flex-direction: column;
  align-items: center;
  gap: 0.25rem;
  padding: var(--spacing-sm);
  background: var(--bg-card);
  border-radius: var(--radius-sm);
}

.metric-label {
  font-size: 0.75rem;
  color: var(--text-muted);
  font-weight: 500;
  text-transform: uppercase;
  letter-spacing: 1px;
}

.metric-value {
  font-size: 1.25rem;
  color: var(--accent-success);
  font-weight: 700;
}

/* ==========================================
   FOOTER
   ========================================== */

.footer {
  position: relative;
  z-index: 1;
  background: var(--bg-secondary);
  border-top: 1px solid var(--border-color);
  padding: var(--spacing-lg) 0;
  margin-top: var(--spacing-xl);
}

.footer-content {
  max-width: 1400px;
  margin: 0 auto;
  padding: 0 var(--spacing-lg);
  text-align: center;
}

.footer-text {
  display: flex;
  align-items: center;
  justify-content: center;
  gap: var(--spacing-sm);
  font-size: 1rem;
  color: var(--text-secondary);
  margin-bottom: 0.5rem;
}

.footer-icon {
  font-size: 1.25rem;
  animation: pulse 2s ease-in-out infinite;
}

.footer-subtext {
  font-size: 0.875rem;
  color: var(--text-muted);
}

/* ==========================================
   ANIMATIONS
   ========================================== */

@keyframes fadeIn {
  from {
    opacity: 0;
  }
  to {
    opacity: 1;
  }
}

@keyframes fadeInUp {
  from {
    opacity: 0;
    transform: translateY(30px);
  }
  to {
    opacity: 1;
    transform: translateY(0);
  }
}

@keyframes fadeInLeft {
  from {
    opacity: 0;
    transform: translateX(-30px);
  }
  to {
    opacity: 1;
    transform: translateX(0);
  }
}

@keyframes fadeInRight {
  from {
    opacity: 0;
    transform: translateX(30px);
  }
  to {
    opacity: 1;
    transform: translateX(0);
  }
}

@keyframes slideInLeft {
  from {
    opacity: 0;
    transform: translateX(-50px);
  }
  to {
    opacity: 1;
    transform: translateX(0);
  }
}

@keyframes slideInRight {
  from {
    opacity: 0;
    transform: translateX(50px);
  }
  to {
    opacity: 1;
    transform: translateX(0);
  }
}

/* ==========================================
   RESPONSIVE DESIGN
   ========================================== */

@media (max-width: 1024px) {
  .hero-section {
    grid-template-columns: 1fr;
    gap: var(--spacing-lg);
  }

  .content-grid {
    grid-template-columns: 1fr;
  }

  .comparison-container {
    grid-template-columns: 1fr;
  }

  .comparison-arrow {
    flex-direction: row;
    justify-content: center;
  }

  .arrow-icon {
    transform: rotate(90deg);
  }
}

@media (max-width: 768px) {
  .hero-title {
    font-size: 2.5rem;
  }

  .hero-subtitle {
    font-size: 1rem;
  }

  .section-title {
    font-size: 2rem;
  }

  .page-title {
    font-size: 2rem;
  }

  .nav-container {
    flex-direction: column;
    gap: var(--spacing-md);
  }

  .nav-links {
    width: 100%;
    justify-content: center;
  }

  .workflow-steps {
    flex-direction: column;
  }

  .workflow-connector {
    width: 2px;
    height: 30px;
    background: linear-gradient(180deg, var(--accent-primary), transparent);
    margin: 0 auto;
  }

  .features-grid {
    grid-template-columns: 1fr;
  }

  .details-grid {
    grid-template-columns: 1fr;
  }

  .metrics-grid {
    grid-template-columns: 1fr;
  }

  .action-section {
    flex-direction: column;
  }

  .action-section .btn {
    width: 100%;
  }
}

@media (max-width: 480px) {
  .main-container {
    padding: var(--spacing-md);
  }

  .hero-title {
    font-size: 2rem;
  }

  .page-title {
    font-size: 1.75rem;
  }

  .section-title {
    font-size: 1.5rem;
  }

  .flash-container {
    right: var(--spacing-sm);
    left: var(--spacing-sm);
    max-width: none;
  }

  .upload-zone {
    padding: var(--spacing-md);
    min-height: 250px;
  }

  .upload-icon-large {
    font-size: 3rem;
  }
}


🔌 9️⃣ Kill Any Previous Servers + Run Flask

## 🔌 Restart Services & Launch Flask App

This cleans any previous Flask/ngrok processes and launches:
- Flask server (background)
- ngrok public URL

After running this section, you will receive a public link to your app.


In [ ]:

# ===============================
# 6️⃣ Kill any previous processes
# ===============================
!pkill -f flask || echo "No flask running"
!pkill -f ngrok || echo "No ngrok running"

In [ ]:
!lsof -i :8000

In [ ]:
!kill -9 35515

In [ ]:
# ===============================
# 7️⃣ Run Flask in the background
# ===============================
!nohup python app.py > flask.log 2>&1 &


🌍 🔟 Start ngrok Tunnel

## 🌍 Generate Public URL using ngrok

Replace the token with your own ngrok token before running.


In [ ]:
# ===============================
# 8️⃣ Start ngrok tunnel
# ===============================
from pyngrok import ngrok, conf
conf.get_default().auth_token = "<YOUR_TOKEN_HERE>"  # 🔑 replace with your token

public_url = ngrok.connect(8000)
print("🌍 Public URL:", public_url)

# ===============================
# 9️⃣ Check logs (optional)
# ===============================
!sleep 3 && tail -n 20 flask.log

📄 1️⃣1️⃣ View Flask Logs

## 📄 Check Flask Logs (Optional)

Useful for debugging if the app does not open or crashes.


In [ ]:
!tail -n 50 flask.log